In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
from random import seed

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import  MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import Imputer

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, LSTM, BatchNormalization
from keras import layers
from keras import optimizers
from keras import callbacks
from keras import models

Using TensorFlow backend.


In [2]:
#Generate Dataset
#Airfoil Self-Noise
df_airfoil=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Airfoil_self_noise.csv')

#Automobile
df_automobile=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Automobile_data.txt')

#Autompg
df_autompg=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Auto-mpg.csv')

#ChallengerUSA
df_challenger=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Challenger USA Space Shuttle O-Ring.csv')

#Combine Cycle Power
df_combine=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Combine Cycle Power Plant.csv')

#Computer Hardware
df_computer=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Computer Hardware.csv')

#Concrete Compressive Strength
df_concrete_compressive_strength=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Compressive Strength.csv')

#Concrete Slump Test
df_concrete1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')
df_concrete3=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Concrete Slump Test.csv')

#Energy Efficiency
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')

#Fertility Diagnosis
df_fertility=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\fertility_Diagnosis.csv')

#Forest Fires
df_forest=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\forestfires.csv')

#Housing
df_housing=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Housing.csv')

#Istanbul Stock Exchange
df_istanbul=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Istanbul Stock Exchange.csv')

#WDBC
df_wdbc=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\wdbc.csv')

In [3]:
#remove Nan columns of df_energy1
df_energy1=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy1.drop(df_energy1.columns[11],inplace=True, axis = 1)
df_energy1.drop(df_energy1.columns[10],inplace=True, axis = 1)

#remove Nan columns of df_energy2
df_energy2=pd.read_csv('C:\\Users\\user\\Desktop\\Thesis\\Datasets\\Energy Efficiency.csv')
df_energy2.drop(df_energy2.columns[11],inplace=True, axis = 1)
df_energy2.drop(df_energy2.columns[10],inplace=True, axis = 1)

#remove question marks
#df_automobile
imp= Imputer(missing_values="NaN", strategy="mean")
df_automobile=df_automobile.replace("?", np.NaN)
df_automobile['num-of-doors'] = df_automobile['num-of-doors'].fillna("four")
for i in df_automobile.columns:
    if df_automobile[i].isnull().sum()>0:
        df_automobile[i]=imp.fit_transform(df_automobile[[i]])
#df_automobile = df_automobile[(df_automobile.astype(str) != '?').all(axis=1)]
#df_automobile.reset_index(drop=True, inplace=True)
df_automobile = df_automobile.drop(['engine-location'], axis=1)
#df_autompg
df_autompg = df_autompg[(df_autompg.astype(str) != '?').all(axis=1)]
df_autompg.reset_index(drop=True, inplace=True)

#convert string to number
le=LabelEncoder()
#df_automobile
string_automobile=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=string_automobile.apply(le.fit_transform)
encoder=df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']].astype(np.int64)
df_automobile[['make','fuel-type','aspiration','num-of-doors', 'body-style', 'drive-wheels', 'engine-type', 'num-of-cylinders', 'fuel-system']]=encoder
a=df_automobile[['normalized-losses','horsepower','price']].astype(np.int64)
b=df_automobile[['bore','stroke','peak-rpm']].astype(float)
df_automobile[['normalized-losses','horsepower','price']]=a
df_automobile[['bore','stroke','peak-rpm']]=b
#df_computer
string_computer=df_computer[['Vendor','Model']]
df_computer[['Vendor','Model']]=string_computer.apply(le.fit_transform)
df_computer[['Vendor','Model']]=df_computer[['Vendor','Model']].astype(np.int64)

#dropna spesific column & special preprocessing
#df_forest
df_forest=df_forest[(df_forest['area'].astype(int) != 0)] 
df_forest.reset_index(drop=True, inplace=True)
num_features=10
dimension=(num_features+1)*9+10
population_size=(2*dimension+1)*2
df_forest=df_forest[:population_size]

C:\Users\user\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
def preprocessing_data1(df,num_features):
    dimension=(num_features+1)*9+10
    population_size=(2*dimension+1)*2
    df=df.dropna()
    df=df[:population_size]
    df
    return df

In [5]:
def feature_selection(df):
    cor=df.corr()
    plt.figure(figsize=(100,50))
    sns.set(font_scale=3)
    sns.heatmap(cor, annot=True, cmap=plt.cm.Blues)
    plt.show()
    return cor

In [6]:
def train_test_split1(x, y):
    x_train, x_test, y_train, y_test= train_test_split(x, y, test_size=0.15, random_state=42)
    return x_train, x_test, y_train, y_test

In [7]:
def preprocessing_data2(df):
    train_size = 0.85*len(df)
    train_size = int(train_size)
    df_train = df[:train_size]
    df_test = df[train_size:]
    print(len(df_test))
    print(len(df_train))
    validation_size=len(df_test)/len(df_train)
    print(validation_size)
    return df_train, df_test, validation_size 

In [8]:
#df_airfoil=preprocessing_data1(df_airfoil,5)
#df_automobile=preprocessing_data1(df_automobile,22)
#df_autompg=preprocessing_data1(df_autompg,7)
#df_challenger=preprocessing_data1(df_challenger,3)
#df_combine=preprocessing_data1(df_combine,4)
#df_computer=preprocessing_data1(df_computer,9)
#df_concrete_compressive_strength=preprocessing_data1(df_concrete_compressive_strength,8)
#df_concrete1=preprocessing_data1(df_concrete1,7)
#df_concrete2=preprocessing_data1(df_concrete2,7)
#df_concrete3=preprocessing_data1(df_concrete3,7)
#df_energy1=preprocessing_data1(df_energy1,8)
#df_energy2=preprocessing_data1(df_energy2,8)

#df_fertility=preprocessing_data1(df_fertility,9)
for i in df_fertility:
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'N'] = 1
    df_fertility.Diagnosis[df_fertility.Diagnosis == 'O'] = 2
df_fertility['Diagnosis']=df_fertility['Diagnosis'].astype(np.int64)

#df_forest=preprocessing_data1(df_forest,10)
#df_housing=preprocessing_data1(df_housing,13)

#df_istanbul=preprocessing_data1(df_istanbul,7)
new_header = df_istanbul.iloc[0]
df_istanbul = df_istanbul[1:]
df_istanbul.columns = new_header
df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']]=df_istanbul[['ISE1','ISE2','SP','DAX','FTSE','NIKKEI','BOVESPA','EU','EM']].astype(np.float64)


#df_wdbc=preprocessing_data1(df_wdbc,10)
df_wdbc.drop(df_wdbc.columns[0], axis=1, inplace = True)
df_wdbc.columns=['id','diagnosis','radius_mean','texture_mean','perimeter_mean','area_mean','smoothness_mean','compactness_mean','concavity_mean','concave points_mean','symmetry_mean','fractal_dimension_mean','radius_se','texture_se','perimeter_se','area_se','smoothness_se','compactness_se','concavity_se','concave points_se','symmetry_se','fractal_dimension_se','radius_worst','texture_worst','perimeter_worst','area_worst','smoothness_worst','compactness_worst','concavity_worst','concave points_worst','symmetry_worst','fractal_dimension_worst']
for i in df_wdbc:
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'M'] = 1
    df_wdbc.diagnosis[df_wdbc.diagnosis == 'B'] = 2
df_wdbc['diagnosis']=df_wdbc['diagnosis'].astype(np.object)

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a 

In [9]:
print(df_airfoil.head())
print(df_autompg.head())
print(df_automobile.head())
print(df_challenger.head())

   Frequency  Angle of Attack  Chord length  Free-stream velocity  \
0        800              0.0        0.3048                  71.3   
1       1000              0.0        0.3048                  71.3   
2       1250              0.0        0.3048                  71.3   
3       1600              0.0        0.3048                  71.3   
4       2000              0.0        0.3048                  71.3   

   Suction side displacement thickness  Scaled sound pressure level  
0                             0.002663                      126.201  
1                             0.002663                      125.201  
2                             0.002663                      125.951  
3                             0.002663                      127.591  
4                             0.002663                      127.461  
    Mpg  Cylinders  Displacement Horsepower  Weight  Acceleration  Model Year  \
0  18.0          8         307.0        130    3504          12.0          70   
1  

In [10]:
#Train and Test 17 datasets
#Airfoil
x_airfoil = df_airfoil.drop(['Scaled sound pressure level'], axis = 1)
y_airfoil = df_airfoil['Scaled sound pressure level']
#Wdbc
x_wdbc = df_wdbc[['radius_mean','perimeter_mean','area_mean','concavity_mean','concave points_mean','radius_worst','perimeter_worst','area_worst','concave points_worst']]
y_wdbc = df_wdbc['diagnosis']
labelencoder_Y = LabelEncoder()
y_wdbc = labelencoder_Y.fit_transform(y_wdbc)
#Automobile
x_automobile = df_automobile.drop(['price','make','body-style'], axis = 1)
y_automobile = df_automobile['price']
y_automobile = np.reshape(y_automobile.values, (-1,1))
#Autompg
x_autompg = df_autompg.drop(['Mpg','Car Name'], axis = 1)
y_autompg = df_autompg['Mpg']
#Challenger
x_challenger = df_challenger.drop(['Number of O-rings','Temporal Order of Flight'], axis = 1)
y_challenger = df_challenger['Number of O-rings']
#Combine
x_combine = df_combine.drop(['PE'], axis = 1)
y_combine = df_combine['PE']
y_combine = np.reshape(y_combine.values, (-1,1))
#Computer
x_computer = df_computer.drop(['ERP'], axis = 1)
y_computer = df_computer['ERP']
y_computer = np.reshape(y_computer.values, (-1,1))
#Concrete_compressive_strength
x_concrete_compressive_strength = df_concrete_compressive_strength.drop(['CCS'], axis = 1)
y_concrete_compressive_strength = df_concrete_compressive_strength['CCS']
y_concrete_compressive_strength = np.reshape(y_concrete_compressive_strength.values, (-1,1))
#Concrete1
x_concrete1 = df_concrete1.drop(['No','Compressive Strength (28-day)(Mpa)'], axis = 1)
y_concrete1 = df_concrete1['Compressive Strength (28-day)(Mpa)']
#Concrete2
x_concrete2 = df_concrete2.drop(['No','SLUMP(cm)'], axis = 1)
y_concrete2 = df_concrete2['SLUMP(cm)']
y_concrete2 = np.reshape(y_concrete2.values, (-1,1))
#Concrete3
x_concrete3 = df_concrete3.drop(['No','FLOW(cm)'], axis = 1)
y_concrete3 = df_concrete3['FLOW(cm)']
y_concrete3 = np.reshape(y_concrete3.values, (-1,1))
#Energy1
x_energy1 = df_energy1.drop(['Y1'], axis = 1)
y_energy1 = df_energy1['Y1']
#Energy2
x_energy2 = df_energy2.drop(['Y2'], axis = 1)
y_energy2 = df_energy2['Y2']
#Fertility
x_fertility = df_fertility.drop(['Diagnosis'], axis = 1)
y_fertility = df_fertility['Diagnosis']
#Forest
x_forest = df_forest.drop(['month','day','area'], axis = 1)
y_forest = df_forest['area']
y_forest = np.reshape(y_forest.values, (-1,1))
#Housing
x_housing = df_housing.drop(['MEDV'], axis = 1)
y_housing = df_housing['MEDV']
#Istanbul
x_istanbul = df_istanbul.drop(['date','ISE1','ISE2'], axis = 1)
y_istanbul = df_istanbul[['ISE1','ISE2']]
y_istanbul['mean'] = y_istanbul.mean(axis=1)
y_istanbul = y_istanbul['mean']

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [11]:
x_train_airfoil, x_test_airfoil, y_train_airfoil, y_test_airfoil = train_test_split1(x_airfoil, y_airfoil)
x_train_automobile, x_test_automobile, y_train_automobile, y_test_automobile = train_test_split1(x_automobile, y_automobile)
x_train_autompg, x_test_autompg, y_train_autompg, y_test_autompg = train_test_split1(x_autompg, y_autompg)
x_train_challenger, x_test_challenger, y_train_challenger, y_test_challenger = train_test_split1(x_challenger, y_challenger)
x_train_combine, x_test_combine, y_train_combine, y_test_combine = train_test_split1(x_combine, y_combine)
x_train_computer, x_test_computer, y_train_computer, y_test_computer = train_test_split1(x_computer, y_computer)
x_train_concrete_compressive_strength, x_test_concrete_compressive_strength, y_train_concrete_compressive_strength, y_test_concrete_compressive_strength = train_test_split1(x_concrete_compressive_strength, y_concrete_compressive_strength)
x_train_concrete1, x_test_concrete1, y_train_concrete1, y_test_concrete1 = train_test_split1(x_concrete1, y_concrete1)
x_train_concrete2, x_test_concrete2, y_train_concrete2, y_test_concrete2 = train_test_split1(x_concrete2, y_concrete2)
x_train_concrete3, x_test_concrete3, y_train_concrete3, y_test_concrete3 = train_test_split1(x_concrete3, y_concrete3)
x_train_energy1, x_test_energy1, y_train_energy1, y_test_energy1 = train_test_split1(x_energy1, y_energy1)
x_train_energy2, x_test_energy2, y_train_energy2, y_test_energy2 = train_test_split1(x_energy2, y_energy2)
x_train_fertility, x_test_fertility, y_train_fertility, y_test_fertility = train_test_split1(x_fertility, y_fertility)
x_train_forest, x_test_forest, y_train_forest, y_test_forest = train_test_split1(x_forest, y_forest)
x_train_housing, x_test_housing, y_train_housing, y_test_housing = train_test_split1(x_housing, y_housing)
x_train_istanbul, x_test_istanbul, y_train_istanbul, y_test_istanbul = train_test_split1(x_istanbul, y_istanbul)
x_train_wdbc, x_test_wdbc, y_train_wdbc, y_test_wdbc = train_test_split1(x_wdbc, y_wdbc)

In [12]:
def train_val_split(x_train, y_train):
    X_train, X_val, Y_train, Y_val= train_test_split(x_train, y_train, test_size=0.176, random_state=42)
    return X_train, X_val, Y_train, Y_val

In [13]:
def standard_scaler(X_train, X_val, x_test):
    sc=StandardScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=sc.fit(X_train)
    #X_scale_val=sc.fit(X_val)
    #x_scale_test=sc.fit(x_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    return X_train, X_val, x_test, X_scale_train

In [14]:
def min_max_scaler_automobile(X_train, X_val, x_test, Y_train, Y_val, y_test):
    minmax_x=MinMaxScaler()
    minmax_y=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=minmax_x.fit(X_train)
    #X_scale_val=min_max.fit(X_val)
    #x_scale_test=min_max.fit(x_test)
    Y_scale_train=minmax_y.fit(Y_train)
    #Y_scale_val=min_max.fit(Y_val)
    #y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_train.transform(X_val)
    x_test=X_scale_train.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=Y_scale_train.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, X_scale_train, Y_scale_train

In [15]:
def min_max_scaler(X_train, X_val, x_test, Y_train, Y_val, y_test):
    min_max=MinMaxScaler()
    X_train=X_train.values
    X_val=X_val.values
    x_test=x_test.values
    X_scale_train=min_max.fit(X_train)
    X_scale_val=min_max.fit(X_val)
    x_scale_test=min_max.fit(x_test)
    Y_scale_train=min_max.fit(Y_train)
    Y_scale_val=min_max.fit(Y_val)
    y_scale_test=min_max.fit(y_test)
    X_train=X_scale_train.transform(X_train)
    X_val=X_scale_val.transform(X_val)
    x_test=x_scale_test.transform(x_test)
    Y_train=Y_scale_train.transform(Y_train)
    Y_val=Y_scale_train.transform(Y_val)
    y_test=y_scale_test.transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test, min_max

In [16]:
def normalization(X_train, X_val, x_test, Y_train, Y_val, y_test):
    normalizer=Normalizer()
    X_train=normalizer.fit_transform(X_train)
    X_val=normalizer.fit_transform(X_val)
    x_test=normalizer.fit_transform(x_test)
    Y_train=normalizer.fit_transform(Y_train)
    Y_val=normalizer.fit_transform(Y_val)
    y_test=normalizer.fit_transform(y_test)
    return X_train, X_val, x_test, Y_train, Y_val, y_test

In [17]:
#Optimizer list
adam0 = optimizers.Adam(lr=0.1)
adam1 = optimizers.Adam(lr=0.001)
adam2 = optimizers.Adam(lr=0.01)
adam3 = optimizers.Adam(lr=0.0001)
adam4 = optimizers.Adam(lr=0.00001)
sgd = optimizers.SGD(lr=0.00001, decay=1e-4, momentum=0.9, nesterov=True)
RMSprop= optimizers.RMSprop(lr=0.01,rho=0.9)
adagrad = optimizers.Adagrad(lr=0.01)
adadelta = optimizers.Adadelta(lr=1.0)

In [18]:
def NN_model_structure_regression(X_train, X_val, Y_train, Y_val, adam, size, epochs, x_test, y_test):
    model = Sequential()
    model.add(Dense(units = 5, input_dim = (X_train.shape[1])))
    model.add(Activation('tanh'))
    #model.add(Dropout(0.2))
    model.add(Dense(units = 10))
    model.add(Activation('linear'))
    model.add(Dense(units = 1))
    model.compile(optimizer = adam , loss = 'mse')
    model.summary()
    history=model.fit(X_train,Y_train, batch_size=size,epochs=epochs, validation_data=(X_val, Y_val), shuffle=True)
    evaluate = model.evaluate(x_test, y_test, verbose=2, batch_size=size)
    print(evaluate)
    return model, history, evaluate

In [19]:
X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil = train_val_split(x_train_airfoil, y_train_airfoil)
X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile = train_val_split(x_train_automobile, y_train_automobile)
X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg = train_val_split(x_train_autompg, y_train_autompg)
X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger = train_val_split(x_train_challenger, y_train_challenger)
X_train_combine, X_val_combine, Y_train_combine, Y_val_combine = train_val_split(x_train_combine, y_train_combine)
X_train_computer, X_val_computer, Y_train_computer, Y_val_computer = train_val_split(x_train_computer, y_train_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength = train_val_split(x_train_concrete_compressive_strength, y_train_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, Y_train_concrete1, Y_val_concrete1 = train_val_split(x_train_concrete1, y_train_concrete1)
X_train_concrete2, X_val_concrete2, Y_train_concrete2, Y_val_concrete2 = train_val_split(x_train_concrete2, y_train_concrete2)
X_train_concrete3, X_val_concrete3, Y_train_concrete3, Y_val_concrete3 = train_val_split(x_train_concrete3, y_train_concrete3)
X_train_energy1, X_val_energy1, Y_train_energy1, Y_val_energy1 = train_val_split(x_train_energy1, y_train_energy1)
X_train_energy2, X_val_energy2, Y_train_energy2, Y_val_energy2 = train_val_split(x_train_energy2, y_train_energy2)
X_train_fertility, X_val_fertility, Y_train_fertility, Y_val_fertility = train_val_split(x_train_fertility, y_train_fertility)
X_train_forest, X_val_forest, Y_train_forest, Y_val_forest = train_val_split(x_train_forest, y_train_forest)
X_train_housing, X_val_housing, Y_train_housing, Y_val_housing = train_val_split(x_train_housing, y_train_housing)
X_train_istanbul, X_val_istanbul, Y_train_istanbul, Y_val_istanbul = train_val_split(x_train_istanbul, y_train_istanbul)
X_train_wdbc, X_val_wdbc, Y_train_wdbc, Y_val_wdbc = train_val_split(x_train_wdbc, y_train_wdbc)

In [20]:
X_train_airfoil, X_val_airfoil, x_test_airfoil, X_scale_train_airfoil=standard_scaler(X_train_airfoil, X_val_airfoil, x_test_airfoil)
#X_train_automobile, X_val_automobile, x_test_automobile=standard_scaler(X_train_automobile, X_val_automobile, x_test_automobile)
#X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile = normalization(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile, X_scale_train_automobile, Y_scale_train_automobile=min_max_scaler_automobile(X_train_automobile, X_val_automobile, x_test_automobile, Y_train_automobile, Y_val_automobile, y_test_automobile)
X_train_autompg, X_val_autompg, x_test_autompg, X_scale_train_autompg=standard_scaler(X_train_autompg, X_val_autompg, x_test_autompg)
X_train_challenger, X_val_challenger, x_test_challenger, X_scale_train_challenger=standard_scaler(X_train_challenger, X_val_challenger, x_test_challenger)
X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine, min_max_combine=min_max_scaler(X_train_combine, X_val_combine, x_test_combine, Y_train_combine, Y_val_combine, y_test_combine)
X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer, min_max_computer=min_max_scaler(X_train_computer, X_val_computer, x_test_computer, Y_train_computer, Y_val_computer, y_test_computer)
X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength, min_max_concrete_compressive_strength=min_max_scaler(X_train_concrete_compressive_strength, X_val_concrete_compressive_strength, x_test_concrete_compressive_strength, Y_train_concrete_compressive_strength, Y_val_concrete_compressive_strength, y_test_concrete_compressive_strength)
X_train_concrete1, X_val_concrete1, x_test_concrete1, X_scale_train_concrete1=standard_scaler(X_train_concrete1, X_val_concrete1, x_test_concrete1)
X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2, min_max_concrete2=min_max_scaler(X_train_concrete2, X_val_concrete2, x_test_concrete2, Y_train_concrete2, Y_val_concrete2, y_test_concrete2)
X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3, min_max_concrete3=min_max_scaler(X_train_concrete3, X_val_concrete3, x_test_concrete3, Y_train_concrete3, Y_val_concrete3, y_test_concrete3)
X_train_energy1, X_val_energy1, x_test_energy1, X_scale_train_energy1=standard_scaler(X_train_energy1, X_val_energy1, x_test_energy1)
X_train_energy2, X_val_energy2, x_test_energy2, X_scale_train_energy2=standard_scaler(X_train_energy2, X_val_energy2, x_test_energy2)
X_train_fertility, X_val_fertility, x_test_fertility, X_scale_train_fertility=standard_scaler(X_train_fertility, X_val_fertility, x_test_fertility)
X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest, min_max_forest=min_max_scaler(X_train_forest, X_val_forest, x_test_forest, Y_train_forest, Y_val_forest, y_test_forest)
X_train_housing, X_val_housing, x_test_housing, X_scale_train_housing=standard_scaler(X_train_housing, X_val_housing, x_test_housing)
X_train_istanbul, X_val_istanbul, x_test_istanbul, X_scale_train_istanbul=standard_scaler(X_train_istanbul, X_val_istanbul, x_test_istanbul)

In [21]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, adam2, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_1 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_2 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________

Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 420us/step - 

Epoch 65/1000
1052/1052 [==============================] - 0s 85us/step - loss: 21.1829 - val_loss: 25.2356
Epoch 66/1000
1052/1052 [==============================] - 0s 90us/step - loss: 20.5806 - val_loss: 24.7603
Epoch 67/1000
1052/1052 [==============================] - 0s 85us/step - loss: 20.3824 - val_loss: 24.6392
Epoch 68/1000
1052/1052 [==============================] - 0s 91us/step - loss: 19.9315 - val_loss: 24.4788
Epoch 69/1000
1052/1052 [==============================] - 0s 86us/step - loss: 19.6757 - val_loss: 24.1428
Epoch 70/1000
1052/1052 [==============================] - 0s 86us/step - loss: 19.8446 - val_loss: 23.9535
Epoch 71/1000
1052/1052 [==============================] - 0s 88us/step - loss: 19.1835 - val_loss: 24.4217
Epoch 72/1000
1052/1052 [==============================] - 0s 88us/step - loss: 19.2706 - val_loss: 24.0228
Epoch 73/1000
1052/1052 [==============================] - 0s 88us/step - loss: 19.1056 - val_loss: 24.3468
Epoch 74/1000
1052/1052 [===

1052/1052 [==============================] - 0s 88us/step - loss: 14.9962 - val_loss: 17.3028
Epoch 141/1000
1052/1052 [==============================] - 0s 87us/step - loss: 14.9825 - val_loss: 20.6548
Epoch 142/1000
1052/1052 [==============================] - 0s 90us/step - loss: 15.4370 - val_loss: 16.8122
Epoch 143/1000
1052/1052 [==============================] - 0s 86us/step - loss: 15.2323 - val_loss: 19.4010
Epoch 144/1000
1052/1052 [==============================] - 0s 83us/step - loss: 14.3286 - val_loss: 17.1199
Epoch 145/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.8039 - val_loss: 16.6718
Epoch 146/1000
1052/1052 [==============================] - 0s 86us/step - loss: 14.3275 - val_loss: 17.0968
Epoch 147/1000
1052/1052 [==============================] - 0s 84us/step - loss: 13.8075 - val_loss: 17.2084
Epoch 148/1000
1052/1052 [==============================] - 0s 87us/step - loss: 13.9035 - val_loss: 17.0453
Epoch 149/1000
1052/1052 [========

1052/1052 [==============================] - 0s 86us/step - loss: 10.2314 - val_loss: 12.4930
Epoch 216/1000
1052/1052 [==============================] - 0s 90us/step - loss: 10.3607 - val_loss: 12.7176
Epoch 217/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.3303 - val_loss: 12.8130
Epoch 218/1000
1052/1052 [==============================] - 0s 86us/step - loss: 10.6977 - val_loss: 12.4748
Epoch 219/1000
1052/1052 [==============================] - 0s 87us/step - loss: 10.3313 - val_loss: 12.7448
Epoch 220/1000
1052/1052 [==============================] - 0s 84us/step - loss: 10.2369 - val_loss: 15.0917
Epoch 221/1000
1052/1052 [==============================] - 0s 88us/step - loss: 10.7919 - val_loss: 13.3972
Epoch 222/1000
1052/1052 [==============================] - 0s 100us/step - loss: 10.1603 - val_loss: 12.3522
Epoch 223/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.7808 - val_loss: 12.6873
Epoch 224/1000
1052/1052 [========

1052/1052 [==============================] - 0s 88us/step - loss: 9.7582 - val_loss: 13.4484
Epoch 291/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.8179 - val_loss: 12.1772
Epoch 292/1000
1052/1052 [==============================] - 0s 105us/step - loss: 9.7995 - val_loss: 14.1703
Epoch 293/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.5206 - val_loss: 12.3166
Epoch 294/1000
1052/1052 [==============================] - 0s 97us/step - loss: 10.0416 - val_loss: 13.9973
Epoch 295/1000
1052/1052 [==============================] - 0s 85us/step - loss: 10.8746 - val_loss: 11.5598
Epoch 296/1000
1052/1052 [==============================] - 0s 79us/step - loss: 10.0741 - val_loss: 12.1147
Epoch 297/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.9077 - val_loss: 11.4144
Epoch 298/1000
1052/1052 [==============================] - 0s 91us/step - loss: 9.8057 - val_loss: 12.1838
Epoch 299/1000
1052/1052 [=============

...

Epoch 745/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.9129 - val_loss: 10.5765
Epoch 746/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8752 - val_loss: 11.1720
Epoch 747/1000
1052/1052 [==============================] - 0s 88us/step - loss: 9.8731 - val_loss: 11.6696
Epoch 748/1000
1052/1052 [==============================] - 0s 91us/step - loss: 8.8808 - val_loss: 10.8189
Epoch 749/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.0036 - val_loss: 10.5767
Epoch 750/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8718 - val_loss: 11.5525
Epoch 751/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7337 - val_loss: 12.0573
Epoch 752/1000
1052/1052 [==============================] - 0s 84us/step - loss: 9.6786 - val_loss: 10.7428
Epoch 753/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2469 - val_loss: 12.2670
Epoch 754/1000
1052/1052 [==

1052/1052 [==============================] - 0s 91us/step - loss: 8.7438 - val_loss: 11.2455
Epoch 821/1000
1052/1052 [==============================] - 0s 96us/step - loss: 8.7134 - val_loss: 11.4666
Epoch 822/1000
1052/1052 [==============================] - 0s 94us/step - loss: 9.3446 - val_loss: 10.9787
Epoch 823/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.6197 - val_loss: 10.6678
Epoch 824/1000
1052/1052 [==============================] - 0s 82us/step - loss: 8.9619 - val_loss: 11.2323
Epoch 825/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7002 - val_loss: 11.5809
Epoch 826/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8551 - val_loss: 10.5741
Epoch 827/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.7548 - val_loss: 11.8369
Epoch 828/1000
1052/1052 [==============================] - 0s 92us/step - loss: 9.4709 - val_loss: 10.7573
Epoch 829/1000
1052/1052 [=================

Epoch 896/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9614 - val_loss: 10.1383
Epoch 897/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6338 - val_loss: 11.1691
Epoch 898/1000
1052/1052 [==============================] - 0s 83us/step - loss: 9.1390 - val_loss: 10.7284
Epoch 899/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.8307 - val_loss: 10.5593
Epoch 900/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.7624 - val_loss: 11.5662
Epoch 901/1000
1052/1052 [==============================] - 0s 86us/step - loss: 9.2717 - val_loss: 10.4144
Epoch 902/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9335 - val_loss: 10.2773
Epoch 903/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.9660 - val_loss: 11.4704
Epoch 904/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.8767 - val_loss: 10.0714
Epoch 905/1000
1052/1052 [==

1052/1052 [==============================] - 0s 88us/step - loss: 8.5246 - val_loss: 10.0580
Epoch 972/1000
1052/1052 [==============================] - 0s 84us/step - loss: 8.5952 - val_loss: 10.7451
Epoch 973/1000
1052/1052 [==============================] - 0s 87us/step - loss: 8.6209 - val_loss: 10.6469
Epoch 974/1000
1052/1052 [==============================] - 0s 85us/step - loss: 9.2501 - val_loss: 10.8181
Epoch 975/1000
1052/1052 [==============================] - 0s 90us/step - loss: 8.9684 - val_loss: 12.6000
Epoch 976/1000
1052/1052 [==============================] - 0s 85us/step - loss: 8.9152 - val_loss: 9.9837
Epoch 977/1000
1052/1052 [==============================] - 0s 86us/step - loss: 8.8568 - val_loss: 10.4143
Epoch 978/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.2613 - val_loss: 12.1647
Epoch 979/1000
1052/1052 [==============================] - 0s 87us/step - loss: 9.3593 - val_loss: 13.0724
Epoch 980/1000
1052/1052 [==================

[array([[-1.5961688 , -3.6006153 , -0.18909729, -3.933795  ,  1.0522664 ],
        [ 0.35881877,  0.08211656, -0.26257172,  0.34114894,  0.13741073],
        [ 0.27578136,  0.09264594, -0.37078285, -0.55793333,  0.37737206],
        [ 0.0615519 ,  0.09321787,  0.08592814,  0.15876016, -0.13260473],
        [-0.18115026, -2.2455835 , -0.20941421, -0.452171  ,  1.1600504 ]],
       dtype=float32),
 array([-1.2705213, -5.0634174, -0.1854675, -4.797664 ,  3.2226675],
       dtype=float32),
 array([[-0.9808444 , -0.7647816 , -0.3112274 ,  0.615847  , -1.0096334 ,
         -0.8339291 , -0.40412965, -0.78470594, -1.0249192 , -0.4282521 ],
        [ 1.597721  ,  2.0967634 ,  1.9138441 , -1.4256531 ,  1.717231  ,
          1.5768399 ,  1.6048434 ,  1.6139784 ,  2.1336641 ,  1.6569222 ],
        [-1.4329431 , -1.4877272 , -0.48921087,  0.73444855, -0.765679  ,
         -0.49142256, -1.1565211 , -0.9261735 , -0.66788226, -1.3124042 ],
        [ 1.2024214 ,  2.0747173 ,  1.5491068 , -1.709986  ,  

In [22]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_adam_4th.h5')

In [23]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, sgd, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_3 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_5 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_4 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 167us/step - l

Epoch 67/1000
1052/1052 [==============================] - 0s 74us/step - loss: 20.1263 - val_loss: 24.5755
Epoch 68/1000
1052/1052 [==============================] - 0s 69us/step - loss: 20.0319 - val_loss: 24.6252
Epoch 69/1000
1052/1052 [==============================] - 0s 67us/step - loss: 20.0378 - val_loss: 24.6148
Epoch 70/1000
1052/1052 [==============================] - 0s 71us/step - loss: 20.1085 - val_loss: 24.5932
Epoch 71/1000
1052/1052 [==============================] - 0s 69us/step - loss: 19.9947 - val_loss: 24.5853
Epoch 72/1000
1052/1052 [==============================] - 0s 71us/step - loss: 20.0093 - val_loss: 24.6281
Epoch 73/1000
1052/1052 [==============================] - 0s 68us/step - loss: 19.9308 - val_loss: 24.5837
Epoch 74/1000
1052/1052 [==============================] - 0s 68us/step - loss: 19.9596 - val_loss: 24.5835
Epoch 75/1000
1052/1052 [==============================] - 0s 71us/step - loss: 19.9382 - val_loss: 24.6193
Epoch 76/1000
1052/1052 [===

1052/1052 [==============================] - 0s 68us/step - loss: 18.9082 - val_loss: 24.3046
Epoch 143/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.9240 - val_loss: 24.3663
Epoch 144/1000
1052/1052 [==============================] - 0s 74us/step - loss: 18.8950 - val_loss: 24.3764
Epoch 145/1000
1052/1052 [==============================] - 0s 73us/step - loss: 18.8994 - val_loss: 24.3111
Epoch 146/1000
1052/1052 [==============================] - 0s 68us/step - loss: 18.8967 - val_loss: 24.3189
Epoch 147/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.8756 - val_loss: 24.3531
Epoch 148/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.8319 - val_loss: 24.3399
Epoch 149/1000
1052/1052 [==============================] - 0s 72us/step - loss: 18.8433 - val_loss: 24.3001
Epoch 150/1000
1052/1052 [==============================] - 0s 70us/step - loss: 18.8330 - val_loss: 24.2955
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 71us/step - loss: 18.0661 - val_loss: 23.7152
Epoch 218/1000
1052/1052 [==============================] - 0s 73us/step - loss: 18.0517 - val_loss: 23.7281
Epoch 219/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.0331 - val_loss: 23.6940
Epoch 220/1000
1052/1052 [==============================] - 0s 71us/step - loss: 18.0204 - val_loss: 23.7363
Epoch 221/1000
1052/1052 [==============================] - 0s 71us/step - loss: 17.9958 - val_loss: 23.6570
Epoch 222/1000
1052/1052 [==============================] - 0s 67us/step - loss: 17.9890 - val_loss: 23.6821
Epoch 223/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.9766 - val_loss: 23.6419
Epoch 224/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.9596 - val_loss: 23.6532
Epoch 225/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.9561 - val_loss: 23.6575
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 73us/step - loss: 17.3109 - val_loss: 22.6924
Epoch 293/1000
1052/1052 [==============================] - 0s 70us/step - loss: 17.3040 - val_loss: 22.6882
Epoch 294/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.2959 - val_loss: 22.7080
Epoch 295/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.2781 - val_loss: 22.6986
Epoch 296/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.2715 - val_loss: 22.6366
Epoch 297/1000
1052/1052 [==============================] - 0s 74us/step - loss: 17.2888 - val_loss: 22.6846
Epoch 298/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.2601 - val_loss: 22.6453
Epoch 299/1000
1052/1052 [==============================] - 0s 73us/step - loss: 17.2444 - val_loss: 22.5955
Epoch 300/1000
1052/1052 [==============================] - 0s 72us/step - loss: 17.2532 - val_loss: 22.6292
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 73us/step - loss: 13.7188 - val_loss: 17.5815
Epoch 743/1000
1052/1052 [==============================] - 0s 74us/step - loss: 13.7173 - val_loss: 17.6036
Epoch 744/1000
1052/1052 [==============================] - 0s 72us/step - loss: 13.7082 - val_loss: 17.5756
Epoch 745/1000
1052/1052 [==============================] - 0s 72us/step - loss: 13.7104 - val_loss: 17.5805
Epoch 746/1000
1052/1052 [==============================] - 0s 72us/step - loss: 13.7006 - val_loss: 17.5872
Epoch 747/1000
1052/1052 [==============================] - 0s 68us/step - loss: 13.7169 - val_loss: 17.6109
Epoch 748/1000
1052/1052 [==============================] - 0s 71us/step - loss: 13.6877 - val_loss: 17.5294
Epoch 749/1000
1052/1052 [==============================] - 0s 76us/step - loss: 13.6914 - val_loss: 17.5809
Epoch 750/1000
1052/1052 [==============================] - 0s 67us/step - loss: 13.7003 - val_loss: 17.5506
Epoch 751/1000
1052/1052 [========

1052/1052 [==============================] - 0s 76us/step - loss: 13.3154 - val_loss: 17.1077
Epoch 818/1000
1052/1052 [==============================] - 0s 78us/step - loss: 13.3125 - val_loss: 17.1079
Epoch 819/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.3079 - val_loss: 17.0765
Epoch 820/1000
1052/1052 [==============================] - 0s 71us/step - loss: 13.2949 - val_loss: 17.0655
Epoch 821/1000
1052/1052 [==============================] - 0s 68us/step - loss: 13.2902 - val_loss: 17.0721
Epoch 822/1000
1052/1052 [==============================] - 0s 74us/step - loss: 13.2896 - val_loss: 17.0363
Epoch 823/1000
1052/1052 [==============================] - 0s 70us/step - loss: 13.2760 - val_loss: 17.0482
Epoch 824/1000
1052/1052 [==============================] - 0s 68us/step - loss: 13.2718 - val_loss: 17.0587
Epoch 825/1000
1052/1052 [==============================] - 0s 66us/step - loss: 13.2669 - val_loss: 17.0540
Epoch 826/1000
1052/1052 [========

1052/1052 [==============================] - 0s 69us/step - loss: 12.9139 - val_loss: 16.6234
Epoch 893/1000
1052/1052 [==============================] - 0s 71us/step - loss: 12.9117 - val_loss: 16.5774
Epoch 894/1000
1052/1052 [==============================] - 0s 71us/step - loss: 12.8953 - val_loss: 16.5995
Epoch 895/1000
1052/1052 [==============================] - 0s 72us/step - loss: 12.8909 - val_loss: 16.5387
Epoch 896/1000
1052/1052 [==============================] - 0s 72us/step - loss: 12.8856 - val_loss: 16.5718
Epoch 897/1000
1052/1052 [==============================] - 0s 71us/step - loss: 12.8895 - val_loss: 16.5924
Epoch 898/1000
1052/1052 [==============================] - 0s 73us/step - loss: 12.8774 - val_loss: 16.5501
Epoch 899/1000
1052/1052 [==============================] - 0s 73us/step - loss: 12.8697 - val_loss: 16.5501
Epoch 900/1000
1052/1052 [==============================] - 0s 75us/step - loss: 12.8647 - val_loss: 16.5378
Epoch 901/1000
1052/1052 [========

1052/1052 [==============================] - 0s 73us/step - loss: 12.5433 - val_loss: 16.1456
Epoch 968/1000
1052/1052 [==============================] - 0s 71us/step - loss: 12.5365 - val_loss: 16.1196
Epoch 969/1000
1052/1052 [==============================] - 0s 73us/step - loss: 12.5329 - val_loss: 16.1129
Epoch 970/1000
1052/1052 [==============================] - 0s 74us/step - loss: 12.5364 - val_loss: 16.0699
Epoch 971/1000
1052/1052 [==============================] - 0s 73us/step - loss: 12.5298 - val_loss: 16.1036
Epoch 972/1000
1052/1052 [==============================] - 0s 69us/step - loss: 12.5264 - val_loss: 16.1010
Epoch 973/1000
1052/1052 [==============================] - 0s 67us/step - loss: 12.5130 - val_loss: 16.0925
Epoch 974/1000
1052/1052 [==============================] - 0s 72us/step - loss: 12.5116 - val_loss: 16.0948
Epoch 975/1000
1052/1052 [==============================] - 0s 72us/step - loss: 12.5092 - val_loss: 16.0812
Epoch 976/1000
1052/1052 [========

[array([[-1.7860088 , -0.547399  , -0.18206193,  1.1447119 ,  0.5612367 ],
        [-0.05468532, -0.7320833 , -0.04239535, -0.13386248,  0.23411496],
        [-0.475015  , -0.46610197, -0.25590572,  0.34989074,  0.03716087],
        [ 0.10180421,  0.09249778,  0.27593103, -0.03799843, -0.00934391],
        [-0.22076379, -0.44906273, -0.2450681 ,  0.5002356 ,  0.33811197]],
       dtype=float32),
 array([-0.31743318,  0.21013595,  2.018742  ,  1.8935071 ,  1.5870582 ],
       dtype=float32),
 array([[ 0.6810596 ,  0.19435947, -0.33184138, -0.17054054,  0.2774041 ,
         -0.40704322,  0.24098921, -0.25961542, -0.08791018,  1.0353488 ],
        [ 0.6882836 ,  0.10551952, -0.17652287,  0.12700604,  0.45058215,
          0.17648332,  0.23190835, -0.90741485,  0.07853812,  0.41600972],
        [ 1.1799165 , -0.18928814,  0.17938834, -1.5282763 ,  1.750538  ,
         -1.3069583 ,  1.0096296 , -1.3503089 ,  1.6176054 ,  1.6447333 ],
        [ 0.9116343 ,  0.527254  , -0.35860318, -1.054717

In [24]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_sgd_4th.h5')

In [25]:
model_airfoil, history_airfoil, evaluate_airfoil=NN_model_structure_regression(X_train_airfoil, X_val_airfoil, Y_train_airfoil, Y_val_airfoil, RMSprop, 32, 1000, x_test_airfoil, y_test_airfoil)
model_airfoil.get_weights()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
activation_5 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                60        
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 101
Trainable params: 101
Non-trainable params: 0
_________________________________________________________________
Train on 1052 samples, validate on 225 samples
Epoch 1/1000
1052/1052 [==============================] - 0s 182us/step - l

1052/1052 [==============================] - 0s 79us/step - loss: 24.1487 - val_loss: 26.8067
Epoch 68/1000
1052/1052 [==============================] - 0s 79us/step - loss: 23.7696 - val_loss: 27.3871
Epoch 69/1000
1052/1052 [==============================] - 0s 82us/step - loss: 23.5703 - val_loss: 26.9707
Epoch 70/1000
1052/1052 [==============================] - 0s 81us/step - loss: 23.1304 - val_loss: 28.9144
Epoch 71/1000
1052/1052 [==============================] - 0s 80us/step - loss: 24.0529 - val_loss: 29.0377
Epoch 72/1000
1052/1052 [==============================] - 0s 85us/step - loss: 23.5308 - val_loss: 25.6649
Epoch 73/1000
1052/1052 [==============================] - 0s 82us/step - loss: 23.8632 - val_loss: 27.0498
Epoch 74/1000
1052/1052 [==============================] - 0s 80us/step - loss: 23.9534 - val_loss: 25.3903
Epoch 75/1000
1052/1052 [==============================] - 0s 79us/step - loss: 23.4602 - val_loss: 26.3573
Epoch 76/1000
1052/1052 [=================

1052/1052 [==============================] - 0s 81us/step - loss: 17.5034 - val_loss: 18.8728
Epoch 143/1000
1052/1052 [==============================] - 0s 82us/step - loss: 17.2101 - val_loss: 18.4343
Epoch 144/1000
1052/1052 [==============================] - 0s 81us/step - loss: 16.9409 - val_loss: 19.1442
Epoch 145/1000
1052/1052 [==============================] - 0s 79us/step - loss: 16.9463 - val_loss: 18.6446
Epoch 146/1000
1052/1052 [==============================] - 0s 78us/step - loss: 16.8088 - val_loss: 17.9835
Epoch 147/1000
1052/1052 [==============================] - 0s 81us/step - loss: 16.9286 - val_loss: 18.7600
Epoch 148/1000
1052/1052 [==============================] - 0s 78us/step - loss: 16.5490 - val_loss: 22.6085
Epoch 149/1000
1052/1052 [==============================] - 0s 80us/step - loss: 16.5144 - val_loss: 20.3677
Epoch 150/1000
1052/1052 [==============================] - 0s 76us/step - loss: 16.8780 - val_loss: 18.1309
Epoch 151/1000
1052/1052 [========

1052/1052 [==============================] - 0s 80us/step - loss: 13.9647 - val_loss: 16.4738
Epoch 218/1000
1052/1052 [==============================] - 0s 79us/step - loss: 14.5671 - val_loss: 15.4198
Epoch 219/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.3700 - val_loss: 24.6590
Epoch 220/1000
1052/1052 [==============================] - 0s 77us/step - loss: 13.8314 - val_loss: 16.5588
Epoch 221/1000
1052/1052 [==============================] - 0s 80us/step - loss: 13.9549 - val_loss: 16.8776
Epoch 222/1000
1052/1052 [==============================] - 0s 81us/step - loss: 13.9094 - val_loss: 16.6132
Epoch 223/1000
1052/1052 [==============================] - 0s 77us/step - loss: 14.1719 - val_loss: 16.5368
Epoch 224/1000
1052/1052 [==============================] - 0s 82us/step - loss: 13.5706 - val_loss: 15.7942
Epoch 225/1000
1052/1052 [==============================] - 0s 79us/step - loss: 13.7159 - val_loss: 15.5028
Epoch 226/1000
1052/1052 [========

1052/1052 [==============================] - 0s 79us/step - loss: 12.3747 - val_loss: 13.1504
Epoch 293/1000
1052/1052 [==============================] - 0s 82us/step - loss: 11.8700 - val_loss: 13.4301
Epoch 294/1000
1052/1052 [==============================] - 0s 79us/step - loss: 11.5397 - val_loss: 12.8531
Epoch 295/1000
1052/1052 [==============================] - 0s 78us/step - loss: 12.2153 - val_loss: 14.2406
Epoch 296/1000
1052/1052 [==============================] - 0s 78us/step - loss: 12.6471 - val_loss: 16.9928
Epoch 297/1000
1052/1052 [==============================] - 0s 79us/step - loss: 11.6147 - val_loss: 13.8726
Epoch 298/1000
1052/1052 [==============================] - 0s 80us/step - loss: 11.9792 - val_loss: 13.6197
Epoch 299/1000
1052/1052 [==============================] - 0s 78us/step - loss: 11.7587 - val_loss: 19.0031
Epoch 300/1000
1052/1052 [==============================] - 0s 77us/step - loss: 11.7579 - val_loss: 14.2481
Epoch 301/1000
1052/1052 [========

...

1052/1052 [==============================] - 0s 80us/step - loss: 8.6751 - val_loss: 9.3545
Epoch 746/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.6299 - val_loss: 11.8947
Epoch 747/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2250 - val_loss: 13.2207
Epoch 748/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.1041 - val_loss: 9.8892
Epoch 749/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.9999 - val_loss: 10.3578
Epoch 750/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2955 - val_loss: 12.4673
Epoch 751/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.9030 - val_loss: 9.7086
Epoch 752/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.4933 - val_loss: 10.0292
Epoch 753/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.1707 - val_loss: 9.7463
Epoch 754/1000
1052/1052 [=====================

1052/1052 [==============================] - 0s 77us/step - loss: 9.3767 - val_loss: 9.7620
Epoch 822/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.1103 - val_loss: 10.3335
Epoch 823/1000
1052/1052 [==============================] - 0s 78us/step - loss: 9.4060 - val_loss: 9.5248
Epoch 824/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2683 - val_loss: 11.5620
Epoch 825/1000
1052/1052 [==============================] - 0s 83us/step - loss: 8.6277 - val_loss: 10.4773
Epoch 826/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.0179 - val_loss: 11.3303
Epoch 827/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.3852 - val_loss: 10.4122
Epoch 828/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.1907 - val_loss: 9.7339
Epoch 829/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.8974 - val_loss: 11.6256
Epoch 830/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 79us/step - loss: 9.3785 - val_loss: 10.2078
Epoch 898/1000
1052/1052 [==============================] - 0s 81us/step - loss: 8.9778 - val_loss: 9.3904
Epoch 899/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.8416 - val_loss: 17.4381
Epoch 900/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.4672 - val_loss: 9.1812
Epoch 901/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.8276 - val_loss: 12.2596
Epoch 902/1000
1052/1052 [==============================] - 0s 82us/step - loss: 9.2868 - val_loss: 10.6706
Epoch 903/1000
1052/1052 [==============================] - 0s 80us/step - loss: 9.1570 - val_loss: 10.8533
Epoch 904/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.2380 - val_loss: 12.3924
Epoch 905/1000
1052/1052 [==============================] - 0s 79us/step - loss: 9.3252 - val_loss: 9.7254
Epoch 906/1000
1052/1052 [====================

1052/1052 [==============================] - 0s 79us/step - loss: 8.8823 - val_loss: 11.9815
Epoch 974/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.1457 - val_loss: 9.6507
Epoch 975/1000
1052/1052 [==============================] - 0s 77us/step - loss: 9.1078 - val_loss: 12.7680
Epoch 976/1000
1052/1052 [==============================] - 0s 78us/step - loss: 8.8648 - val_loss: 10.6131
Epoch 977/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.8196 - val_loss: 9.6597
Epoch 978/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.9919 - val_loss: 12.8627
Epoch 979/1000
1052/1052 [==============================] - 0s 81us/step - loss: 9.2773 - val_loss: 9.2700
Epoch 980/1000
1052/1052 [==============================] - 0s 80us/step - loss: 8.9882 - val_loss: 9.7748
Epoch 981/1000
1052/1052 [==============================] - 0s 79us/step - loss: 8.8564 - val_loss: 9.8148
Epoch 982/1000
1052/1052 [======================

[array([[-0.2302649 , -3.97331   ,  0.63084215, -1.9847475 ,  3.2233388 ],
        [-0.27515176,  0.19444647,  1.1118668 ,  0.48282054, -0.05107445],
        [-0.39600155, -0.72132504,  1.3608668 ,  0.4297108 , -0.01219113],
        [ 0.18915212,  0.17059492, -0.21388571,  0.0339576 , -0.07747869],
        [-0.23181646, -0.3591916 , -0.08566522, -0.29420462,  2.5180795 ]],
       dtype=float32),
 array([ 1.5570226 , -4.9057674 ,  0.67366976, -1.2789114 ,  4.863223  ],
       dtype=float32),
 array([[-0.9601603 ,  0.72588295, -1.4677371 ,  1.4341422 ,  1.1197554 ,
         -0.69501907, -1.5342631 ,  0.77839696,  1.0789837 ,  1.5213062 ],
        [ 1.4833688 , -1.9004036 ,  1.2655417 , -2.5006552 , -1.7641214 ,
          2.2216558 ,  2.1828544 , -1.7521191 , -2.472325  , -2.3168318 ],
        [ 0.49374238, -0.50299215,  0.7508968 , -0.47755063,  0.28167087,
          0.4474679 ,  0.5066853 ,  0.08460659, -0.12034156, -0.6250875 ],
        [-0.3407948 ,  0.6176681 , -0.82031447,  0.974647

In [26]:
model_airfoil.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_airfoil_rmsprop_4th.h5')

In [27]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, adam2, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_10 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_7 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_11 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_8 (Activation)    (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 640us/step - loss:

274/274 [==============================] - 0s 106us/step - loss: 7.0347 - val_loss: 9.8262
Epoch 69/1000
274/274 [==============================] - 0s 107us/step - loss: 6.6585 - val_loss: 9.7644
Epoch 70/1000
274/274 [==============================] - 0s 111us/step - loss: 6.5831 - val_loss: 9.2996
Epoch 71/1000
274/274 [==============================] - 0s 102us/step - loss: 6.5821 - val_loss: 9.3149
Epoch 72/1000
274/274 [==============================] - 0s 108us/step - loss: 6.6539 - val_loss: 9.3789
Epoch 73/1000
274/274 [==============================] - 0s 114us/step - loss: 6.5702 - val_loss: 9.7010
Epoch 74/1000
274/274 [==============================] - 0s 106us/step - loss: 6.5268 - val_loss: 9.4154
Epoch 75/1000
274/274 [==============================] - 0s 116us/step - loss: 6.5497 - val_loss: 9.3861
Epoch 76/1000
274/274 [==============================] - 0s 109us/step - loss: 6.5219 - val_loss: 9.3606
Epoch 77/1000
274/274 [==============================] - 0s 109us/ste

Epoch 146/1000
274/274 [==============================] - 0s 105us/step - loss: 6.3614 - val_loss: 9.0932
Epoch 147/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4549 - val_loss: 9.1301
Epoch 148/1000
274/274 [==============================] - 0s 106us/step - loss: 6.2410 - val_loss: 8.9687
Epoch 149/1000
274/274 [==============================] - 0s 114us/step - loss: 6.2604 - val_loss: 9.1778
Epoch 150/1000
274/274 [==============================] - 0s 102us/step - loss: 6.4146 - val_loss: 9.1156
Epoch 151/1000
274/274 [==============================] - 0s 104us/step - loss: 6.4418 - val_loss: 9.4285
Epoch 152/1000
274/274 [==============================] - 0s 100us/step - loss: 6.2414 - val_loss: 9.2058
Epoch 153/1000
274/274 [==============================] - 0s 98us/step - loss: 6.3471 - val_loss: 9.4081
Epoch 154/1000
274/274 [==============================] - 0s 98us/step - loss: 6.2424 - val_loss: 9.0697
Epoch 155/1000
274/274 [========================

274/274 [==============================] - 0s 102us/step - loss: 5.8765 - val_loss: 9.0288
Epoch 224/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7289 - val_loss: 9.1734
Epoch 225/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6970 - val_loss: 9.2256
Epoch 226/1000
274/274 [==============================] - 0s 98us/step - loss: 5.7759 - val_loss: 9.2208
Epoch 227/1000
274/274 [==============================] - 0s 103us/step - loss: 5.7134 - val_loss: 9.0993
Epoch 228/1000
274/274 [==============================] - 0s 103us/step - loss: 5.8429 - val_loss: 9.2171
Epoch 229/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7353 - val_loss: 9.3124
Epoch 230/1000
274/274 [==============================] - 0s 107us/step - loss: 5.6788 - val_loss: 9.3859
Epoch 231/1000
274/274 [==============================] - 0s 106us/step - loss: 5.8326 - val_loss: 9.2654
Epoch 232/1000
274/274 [==============================] - 0s 106

274/274 [==============================] - 0s 102us/step - loss: 5.4866 - val_loss: 9.0781
Epoch 301/1000
274/274 [==============================] - 0s 105us/step - loss: 5.5172 - val_loss: 9.0759
Epoch 302/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6298 - val_loss: 9.0283
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 5.7492 - val_loss: 9.0670
Epoch 304/1000
274/274 [==============================] - 0s 102us/step - loss: 5.8722 - val_loss: 9.3194
Epoch 305/1000
274/274 [==============================] - 0s 99us/step - loss: 5.7718 - val_loss: 9.0129
Epoch 306/1000
274/274 [==============================] - 0s 95us/step - loss: 5.6887 - val_loss: 9.4101
Epoch 307/1000
274/274 [==============================] - 0s 104us/step - loss: 5.7772 - val_loss: 9.1289
Epoch 308/1000
274/274 [==============================] - 0s 103us/step - loss: 5.5854 - val_loss: 9.3151
Epoch 309/1000
274/274 [==============================] - 0s 98u

...

Epoch 762/1000
274/274 [==============================] - 0s 102us/step - loss: 5.6612 - val_loss: 10.5110
Epoch 763/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2869 - val_loss: 10.6499
Epoch 764/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1000 - val_loss: 10.3780
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1723 - val_loss: 10.5457
Epoch 766/1000
274/274 [==============================] - 0s 99us/step - loss: 5.0882 - val_loss: 10.3938
Epoch 767/1000
274/274 [==============================] - 0s 102us/step - loss: 5.2533 - val_loss: 10.4972
Epoch 768/1000
274/274 [==============================] - 0s 97us/step - loss: 5.1421 - val_loss: 10.4340
Epoch 769/1000
274/274 [==============================] - 0s 102us/step - loss: 5.1617 - val_loss: 10.6694
Epoch 770/1000
274/274 [==============================] - 0s 109us/step - loss: 5.1900 - val_loss: 10.5407
Epoch 771/1000
274/274 [================

Epoch 839/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0569 - val_loss: 10.3996
Epoch 840/1000
274/274 [==============================] - 0s 90us/step - loss: 5.0799 - val_loss: 10.5952
Epoch 841/1000
274/274 [==============================] - 0s 98us/step - loss: 5.1122 - val_loss: 10.4059
Epoch 842/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0786 - val_loss: 10.5999
Epoch 843/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0934 - val_loss: 10.7097
Epoch 844/1000
274/274 [==============================] - 0s 109us/step - loss: 5.0828 - val_loss: 10.6486
Epoch 845/1000
274/274 [==============================] - 0s 108us/step - loss: 5.1722 - val_loss: 10.4368
Epoch 846/1000
274/274 [==============================] - 0s 103us/step - loss: 5.0858 - val_loss: 10.6383
Epoch 847/1000
274/274 [==============================] - 0s 95us/step - loss: 5.2344 - val_loss: 10.4806
Epoch 848/1000
274/274 [=================

Epoch 916/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0496 - val_loss: 10.7711
Epoch 917/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0659 - val_loss: 10.6515
Epoch 918/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0851 - val_loss: 10.4574
Epoch 919/1000
274/274 [==============================] - 0s 106us/step - loss: 5.1106 - val_loss: 10.7078
Epoch 920/1000
274/274 [==============================] - 0s 100us/step - loss: 5.2830 - val_loss: 10.5303
Epoch 921/1000
274/274 [==============================] - 0s 95us/step - loss: 5.1110 - val_loss: 10.7695
Epoch 922/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0876 - val_loss: 10.5609
Epoch 923/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0364 - val_loss: 10.8071
Epoch 924/1000
274/274 [==============================] - 0s 91us/step - loss: 5.1353 - val_loss: 10.7497
Epoch 925/1000
274/274 [===============

274/274 [==============================] - 0s 102us/step - loss: 5.1257 - val_loss: 10.8620
Epoch 993/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0605 - val_loss: 10.8669
Epoch 994/1000
274/274 [==============================] - 0s 106us/step - loss: 5.0831 - val_loss: 10.5481
Epoch 995/1000
274/274 [==============================] - 0s 101us/step - loss: 5.0453 - val_loss: 10.9285
Epoch 996/1000
274/274 [==============================] - 0s 91us/step - loss: 5.1022 - val_loss: 10.7318
Epoch 997/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0606 - val_loss: 10.4800
Epoch 998/1000
274/274 [==============================] - 0s 105us/step - loss: 5.0373 - val_loss: 10.7699
Epoch 999/1000
274/274 [==============================] - 0s 98us/step - loss: 5.0259 - val_loss: 10.7002
Epoch 1000/1000
274/274 [==============================] - 0s 102us/step - loss: 5.0232 - val_loss: 10.6545
7.788502499208612


[array([[ 1.0817424 ,  2.9780128 ,  0.8902802 ,  2.1889203 ,  1.4394155 ],
        [-0.4769123 , -2.3045018 ,  0.5825157 , -2.2376568 , -0.64197856],
        [-0.37907726,  2.2984297 , -0.7056434 ,  0.23430349,  0.16449597],
        [-0.41197366,  2.3706481 , -1.7816945 ,  2.6886096 ,  0.80522037],
        [-0.68378866,  0.02428338,  0.01787698, -1.628269  ,  0.44999048],
        [ 0.3437196 , -1.2218002 ,  1.3378663 , -1.2520794 , -1.0812557 ],
        [ 1.4439223 , -0.827776  ,  0.7537029 ,  0.62198   , -1.2509274 ]],
       dtype=float32),
 array([ 1.6387322,  2.5258436, -2.3701496, -2.1621914,  2.8753843],
       dtype=float32),
 array([[-0.06682646,  0.25623688,  0.1394079 ,  0.25296515, -0.23508807,
          0.03978946,  0.2139449 ,  0.66538095, -0.45425263,  0.72813016],
        [ 0.72415364,  0.22388478, -0.38523257, -0.7043643 ,  0.04338961,
         -0.18492943, -0.46247217, -0.628875  , -0.28383246, -0.57251287],
        [-1.0293243 ,  1.2387242 ,  1.1569691 ,  0.09119947, 

In [28]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_adam_4th.h5')

In [29]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, sgd, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_9 (Activation)    (None, 5)                 0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_10 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_15 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 528us/step - loss:

274/274 [==============================] - 0s 84us/step - loss: 461.6372 - val_loss: 491.6884
Epoch 68/1000
274/274 [==============================] - 0s 87us/step - loss: 458.5077 - val_loss: 488.2437
Epoch 69/1000
274/274 [==============================] - 0s 76us/step - loss: 455.2509 - val_loss: 484.7440
Epoch 70/1000
274/274 [==============================] - 0s 87us/step - loss: 451.9789 - val_loss: 481.1447
Epoch 71/1000
274/274 [==============================] - 0s 80us/step - loss: 448.6233 - val_loss: 477.5210
Epoch 72/1000
274/274 [==============================] - 0s 87us/step - loss: 445.2111 - val_loss: 473.8710
Epoch 73/1000
274/274 [==============================] - 0s 87us/step - loss: 441.8422 - val_loss: 470.0860
Epoch 74/1000
274/274 [==============================] - 0s 80us/step - loss: 438.2898 - val_loss: 466.3189
Epoch 75/1000
274/274 [==============================] - 0s 87us/step - loss: 434.7525 - val_loss: 462.5008
Epoch 76/1000
274/274 [===================

274/274 [==============================] - 0s 87us/step - loss: 118.3736 - val_loss: 125.8284
Epoch 143/1000
274/274 [==============================] - 0s 87us/step - loss: 114.3878 - val_loss: 121.6681
Epoch 144/1000
274/274 [==============================] - 0s 76us/step - loss: 110.4922 - val_loss: 117.6609
Epoch 145/1000
274/274 [==============================] - 0s 80us/step - loss: 106.7474 - val_loss: 113.6891
Epoch 146/1000
274/274 [==============================] - 0s 91us/step - loss: 103.0189 - val_loss: 109.8839
Epoch 147/1000
274/274 [==============================] - 0s 80us/step - loss: 99.4495 - val_loss: 106.1803
Epoch 148/1000
274/274 [==============================] - 0s 76us/step - loss: 96.0139 - val_loss: 102.4956
Epoch 149/1000
274/274 [==============================] - 0s 84us/step - loss: 92.5680 - val_loss: 98.9796
Epoch 150/1000
274/274 [==============================] - 0s 80us/step - loss: 89.3217 - val_loss: 95.5660
Epoch 151/1000
274/274 [================

274/274 [==============================] - 0s 87us/step - loss: 21.1486 - val_loss: 23.7643
Epoch 219/1000
274/274 [==============================] - 0s 87us/step - loss: 21.0713 - val_loss: 23.6534
Epoch 220/1000
274/274 [==============================] - 0s 91us/step - loss: 20.9917 - val_loss: 23.5504
Epoch 221/1000
274/274 [==============================] - 0s 84us/step - loss: 20.9117 - val_loss: 23.4576
Epoch 222/1000
274/274 [==============================] - 0s 87us/step - loss: 20.8453 - val_loss: 23.3715
Epoch 223/1000
274/274 [==============================] - 0s 84us/step - loss: 20.7801 - val_loss: 23.2921
Epoch 224/1000
274/274 [==============================] - 0s 98us/step - loss: 20.7196 - val_loss: 23.2151
Epoch 225/1000
274/274 [==============================] - 0s 80us/step - loss: 20.6571 - val_loss: 23.1398
Epoch 226/1000
274/274 [==============================] - 0s 84us/step - loss: 20.6092 - val_loss: 23.0556
Epoch 227/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 18.6645 - val_loss: 20.5482
Epoch 295/1000
274/274 [==============================] - 0s 83us/step - loss: 18.6441 - val_loss: 20.5210
Epoch 296/1000
274/274 [==============================] - 0s 84us/step - loss: 18.6259 - val_loss: 20.5005
Epoch 297/1000
274/274 [==============================] - 0s 87us/step - loss: 18.6069 - val_loss: 20.4831
Epoch 298/1000
274/274 [==============================] - 0s 84us/step - loss: 18.5908 - val_loss: 20.4592
Epoch 299/1000
274/274 [==============================] - 0s 95us/step - loss: 18.5719 - val_loss: 20.4392
Epoch 300/1000
274/274 [==============================] - 0s 84us/step - loss: 18.5540 - val_loss: 20.4110
Epoch 301/1000
274/274 [==============================] - 0s 95us/step - loss: 18.5350 - val_loss: 20.3903
Epoch 302/1000
274/274 [==============================] - 0s 84us/step - loss: 18.5171 - val_loss: 20.3699
Epoch 303/1000
274/274 [============================

...

274/274 [==============================] - 0s 84us/step - loss: 13.7799 - val_loss: 13.9835
Epoch 751/1000
274/274 [==============================] - 0s 91us/step - loss: 13.7739 - val_loss: 13.9734
Epoch 752/1000
274/274 [==============================] - 0s 80us/step - loss: 13.7656 - val_loss: 13.9661
Epoch 753/1000
274/274 [==============================] - 0s 87us/step - loss: 13.7593 - val_loss: 13.9581
Epoch 754/1000
274/274 [==============================] - 0s 84us/step - loss: 13.7529 - val_loss: 13.9485
Epoch 755/1000
274/274 [==============================] - 0s 80us/step - loss: 13.7456 - val_loss: 13.9381
Epoch 756/1000
274/274 [==============================] - 0s 84us/step - loss: 13.7384 - val_loss: 13.9271
Epoch 757/1000
274/274 [==============================] - 0s 84us/step - loss: 13.7326 - val_loss: 13.9174
Epoch 758/1000
274/274 [==============================] - 0s 87us/step - loss: 13.7248 - val_loss: 13.9106
Epoch 759/1000
274/274 [============================

274/274 [==============================] - 0s 82us/step - loss: 13.2825 - val_loss: 13.3210
Epoch 827/1000
274/274 [==============================] - 0s 91us/step - loss: 13.2746 - val_loss: 13.3149
Epoch 828/1000
274/274 [==============================] - 0s 87us/step - loss: 13.2691 - val_loss: 13.3082
Epoch 829/1000
274/274 [==============================] - 0s 93us/step - loss: 13.2624 - val_loss: 13.2994
Epoch 830/1000
274/274 [==============================] - 0s 87us/step - loss: 13.2569 - val_loss: 13.2905
Epoch 831/1000
274/274 [==============================] - 0s 93us/step - loss: 13.2504 - val_loss: 13.2822
Epoch 832/1000
274/274 [==============================] - 0s 87us/step - loss: 13.2457 - val_loss: 13.2734
Epoch 833/1000
274/274 [==============================] - 0s 87us/step - loss: 13.2377 - val_loss: 13.2668
Epoch 834/1000
274/274 [==============================] - 0s 84us/step - loss: 13.2331 - val_loss: 13.2605
Epoch 835/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 12.8343 - val_loss: 12.7574
Epoch 903/1000
274/274 [==============================] - 0s 87us/step - loss: 12.8299 - val_loss: 12.7507
Epoch 904/1000
274/274 [==============================] - 0s 84us/step - loss: 12.8254 - val_loss: 12.7432
Epoch 905/1000
274/274 [==============================] - 0s 84us/step - loss: 12.8179 - val_loss: 12.7368
Epoch 906/1000
274/274 [==============================] - 0s 84us/step - loss: 12.8126 - val_loss: 12.7306
Epoch 907/1000
274/274 [==============================] - 0s 84us/step - loss: 12.8070 - val_loss: 12.7247
Epoch 908/1000
274/274 [==============================] - 0s 84us/step - loss: 12.8021 - val_loss: 12.7181
Epoch 909/1000
274/274 [==============================] - 0s 87us/step - loss: 12.7960 - val_loss: 12.7118
Epoch 910/1000
274/274 [==============================] - 0s 87us/step - loss: 12.7904 - val_loss: 12.7058
Epoch 911/1000
274/274 [============================

274/274 [==============================] - 0s 84us/step - loss: 12.4299 - val_loss: 12.2717
Epoch 979/1000
274/274 [==============================] - 0s 91us/step - loss: 12.4251 - val_loss: 12.2650
Epoch 980/1000
274/274 [==============================] - 0s 80us/step - loss: 12.4196 - val_loss: 12.2590
Epoch 981/1000
274/274 [==============================] - 0s 87us/step - loss: 12.4145 - val_loss: 12.2539
Epoch 982/1000
274/274 [==============================] - 0s 80us/step - loss: 12.4097 - val_loss: 12.2468
Epoch 983/1000
274/274 [==============================] - 0s 84us/step - loss: 12.4052 - val_loss: 12.2409
Epoch 984/1000
274/274 [==============================] - 0s 92us/step - loss: 12.4014 - val_loss: 12.2344
Epoch 985/1000
274/274 [==============================] - 0s 84us/step - loss: 12.3943 - val_loss: 12.2296
Epoch 986/1000
274/274 [==============================] - 0s 76us/step - loss: 12.3909 - val_loss: 12.2238
Epoch 987/1000
274/274 [============================

[array([[ 0.11380088, -0.42094195,  0.7532695 , -0.61172146,  0.5396082 ],
        [-0.11679897,  0.1834787 ,  0.14322117, -0.62558264, -0.22440511],
        [-0.10415284, -0.0633937 , -0.20125364, -0.5518582 ,  0.29965374],
        [ 0.16430552,  0.34165844, -0.32889494,  0.38772848,  0.46921068],
        [-0.10892204,  0.15316896,  0.10988298, -0.37657747,  0.244739  ],
        [-0.32914495, -0.15095642,  0.297942  ,  0.17218754,  0.1948211 ],
        [-0.00881684, -0.12983917,  0.05096454,  0.4963237 , -0.42085904]],
       dtype=float32),
 array([-0.5659236 , -1.2312715 ,  0.70134324,  0.4800585 , -0.22665653],
       dtype=float32),
 array([[ 0.11289884, -0.69982374, -0.47021228,  0.19327365, -0.98451114,
          0.74613345, -0.3855398 , -0.53793913, -0.29389274,  0.47359297],
        [ 0.8753245 , -0.25687602,  0.18520978,  0.41818953, -0.44019747,
          1.2134038 ,  0.69230264, -0.5812103 ,  0.34436142,  0.717147  ],
        [-0.6743195 ,  0.83480436,  0.01753288,  0.32758

In [30]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_sgd_4th.h5')

In [31]:
model_autompg, history_autompg, evaluate_autompg=NN_model_structure_regression(X_train_autompg, X_val_autompg, Y_train_autompg, Y_val_autompg, RMSprop, 32, 1000, x_test_autompg, y_test_autompg)
model_autompg.get_weights()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 5)                 40        
_________________________________________________________________
activation_11 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_17 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_12 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 11        
Total params: 111
Trainable params: 111
Non-trainable params: 0
_________________________________________________________________
Train on 274 samples, validate on 59 samples
Epoch 1/1000
274/274 [==============================] - 0s 587us/step - loss:

274/274 [==============================] - 0s 102us/step - loss: 7.9491 - val_loss: 11.1131
Epoch 69/1000
274/274 [==============================] - 0s 95us/step - loss: 8.2887 - val_loss: 12.2274
Epoch 70/1000
274/274 [==============================] - 0s 87us/step - loss: 8.1602 - val_loss: 10.3124
Epoch 71/1000
274/274 [==============================] - 0s 91us/step - loss: 7.9453 - val_loss: 11.0328
Epoch 72/1000
274/274 [==============================] - 0s 120us/step - loss: 7.8296 - val_loss: 10.8918
Epoch 73/1000
274/274 [==============================] - 0s 97us/step - loss: 7.2667 - val_loss: 10.5710
Epoch 74/1000
274/274 [==============================] - 0s 87us/step - loss: 8.1486 - val_loss: 10.6020
Epoch 75/1000
274/274 [==============================] - 0s 87us/step - loss: 8.1316 - val_loss: 10.5330
Epoch 76/1000
274/274 [==============================] - 0s 98us/step - loss: 7.9547 - val_loss: 10.4985
Epoch 77/1000
274/274 [==============================] - 0s 87us/st

274/274 [==============================] - 0s 101us/step - loss: 7.9495 - val_loss: 10.4724
Epoch 146/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8034 - val_loss: 10.3985
Epoch 147/1000
274/274 [==============================] - 0s 95us/step - loss: 7.7539 - val_loss: 11.3069
Epoch 148/1000
274/274 [==============================] - 0s 87us/step - loss: 8.0025 - val_loss: 11.4638
Epoch 149/1000
274/274 [==============================] - 0s 98us/step - loss: 7.6788 - val_loss: 10.6381
Epoch 150/1000
274/274 [==============================] - 0s 91us/step - loss: 7.4644 - val_loss: 10.3505
Epoch 151/1000
274/274 [==============================] - 0s 97us/step - loss: 8.1472 - val_loss: 10.3285
Epoch 152/1000
274/274 [==============================] - 0s 87us/step - loss: 7.4156 - val_loss: 11.0320
Epoch 153/1000
274/274 [==============================] - 0s 95us/step - loss: 7.5627 - val_loss: 10.6087
Epoch 154/1000
274/274 [==============================] - 0s

274/274 [==============================] - 0s 91us/step - loss: 7.3618 - val_loss: 12.2057
Epoch 223/1000
274/274 [==============================] - 0s 94us/step - loss: 7.4042 - val_loss: 10.8368
Epoch 224/1000
274/274 [==============================] - 0s 102us/step - loss: 7.9449 - val_loss: 12.7899
Epoch 225/1000
274/274 [==============================] - 0s 89us/step - loss: 7.6102 - val_loss: 11.1360
Epoch 226/1000
274/274 [==============================] - 0s 102us/step - loss: 7.6872 - val_loss: 11.6701
Epoch 227/1000
274/274 [==============================] - 0s 91us/step - loss: 7.2570 - val_loss: 10.3954
Epoch 228/1000
274/274 [==============================] - 0s 94us/step - loss: 7.0123 - val_loss: 9.9633
Epoch 229/1000
274/274 [==============================] - 0s 95us/step - loss: 7.3968 - val_loss: 10.4457
Epoch 230/1000
274/274 [==============================] - 0s 95us/step - loss: 7.8930 - val_loss: 12.8510
Epoch 231/1000
274/274 [==============================] - 0s

274/274 [==============================] - 0s 91us/step - loss: 7.3178 - val_loss: 11.5167
Epoch 300/1000
274/274 [==============================] - 0s 95us/step - loss: 7.6300 - val_loss: 10.5697
Epoch 301/1000
274/274 [==============================] - 0s 91us/step - loss: 8.0434 - val_loss: 10.6898
Epoch 302/1000
274/274 [==============================] - 0s 87us/step - loss: 7.1930 - val_loss: 9.9564
Epoch 303/1000
274/274 [==============================] - 0s 95us/step - loss: 7.7208 - val_loss: 10.6082
Epoch 304/1000
274/274 [==============================] - 0s 91us/step - loss: 7.3634 - val_loss: 10.3165
Epoch 305/1000
274/274 [==============================] - 0s 88us/step - loss: 7.1431 - val_loss: 10.5518
Epoch 306/1000
274/274 [==============================] - 0s 90us/step - loss: 7.6994 - val_loss: 10.2090
Epoch 307/1000
274/274 [==============================] - 0s 88us/step - loss: 7.3865 - val_loss: 10.6076
Epoch 308/1000
274/274 [==============================] - 0s 9

...

274/274 [==============================] - 0s 85us/step - loss: 5.6520 - val_loss: 10.7371
Epoch 762/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0148 - val_loss: 10.4813
Epoch 763/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8483 - val_loss: 10.5429
Epoch 764/1000
274/274 [==============================] - 0s 87us/step - loss: 5.8226 - val_loss: 11.6604
Epoch 765/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8537 - val_loss: 10.2465
Epoch 766/1000
274/274 [==============================] - ETA: 0s - loss: 8.935 - 0s 91us/step - loss: 5.8165 - val_loss: 10.0162
Epoch 767/1000
274/274 [==============================] - 0s 84us/step - loss: 5.8716 - val_loss: 11.1794
Epoch 768/1000
274/274 [==============================] - 0s 95us/step - loss: 6.0591 - val_loss: 10.1749
Epoch 769/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6067 - val_loss: 10.2381
Epoch 770/1000
274/274 [=============

274/274 [==============================] - 0s 102us/step - loss: 5.8718 - val_loss: 10.3083
Epoch 838/1000
274/274 [==============================] - 0s 102us/step - loss: 5.7278 - val_loss: 10.0578
Epoch 839/1000
274/274 [==============================] - 0s 91us/step - loss: 5.9379 - val_loss: 9.9283
Epoch 840/1000
274/274 [==============================] - 0s 87us/step - loss: 5.7006 - val_loss: 10.9383
Epoch 841/1000
274/274 [==============================] - 0s 86us/step - loss: 5.7660 - val_loss: 11.0818
Epoch 842/1000
274/274 [==============================] - 0s 93us/step - loss: 5.7113 - val_loss: 11.8104
Epoch 843/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8696 - val_loss: 11.1272
Epoch 844/1000
274/274 [==============================] - 0s 98us/step - loss: 5.8025 - val_loss: 10.0835
Epoch 845/1000
274/274 [==============================] - 0s 91us/step - loss: 5.9933 - val_loss: 9.9538
Epoch 846/1000
274/274 [==============================] - 0s 

274/274 [==============================] - 0s 91us/step - loss: 5.4187 - val_loss: 9.9308
Epoch 915/1000
274/274 [==============================] - 0s 84us/step - loss: 5.3926 - val_loss: 11.9036
Epoch 916/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6413 - val_loss: 9.4011
Epoch 917/1000
274/274 [==============================] - 0s 87us/step - loss: 5.9212 - val_loss: 10.5113
Epoch 918/1000
274/274 [==============================] - 0s 92us/step - loss: 5.4440 - val_loss: 11.0352
Epoch 919/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8266 - val_loss: 11.3423
Epoch 920/1000
274/274 [==============================] - 0s 95us/step - loss: 5.8738 - val_loss: 12.8338
Epoch 921/1000
274/274 [==============================] - 0s 91us/step - loss: 5.6881 - val_loss: 10.3660
Epoch 922/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3103 - val_loss: 10.1912
Epoch 923/1000
274/274 [==============================] - 0s 10

274/274 [==============================] - 0s 91us/step - loss: 5.6248 - val_loss: 9.8954
Epoch 992/1000
274/274 [==============================] - 0s 98us/step - loss: 5.6656 - val_loss: 11.1691
Epoch 993/1000
274/274 [==============================] - 0s 91us/step - loss: 5.9644 - val_loss: 10.3481
Epoch 994/1000
274/274 [==============================] - 0s 87us/step - loss: 5.3845 - val_loss: 11.2993
Epoch 995/1000
274/274 [==============================] - 0s 91us/step - loss: 5.3191 - val_loss: 10.7927
Epoch 996/1000
274/274 [==============================] - 0s 84us/step - loss: 5.4590 - val_loss: 9.8063
Epoch 997/1000
274/274 [==============================] - 0s 87us/step - loss: 5.6328 - val_loss: 13.2779
Epoch 998/1000
274/274 [==============================] - 0s 87us/step - loss: 5.7746 - val_loss: 9.6389
Epoch 999/1000
274/274 [==============================] - 0s 91us/step - loss: 5.5276 - val_loss: 10.1473
Epoch 1000/1000
274/274 [==============================] - 0s 91

[array([[ 2.2319605 ,  1.9935802 , -2.0282888 , -6.257301  ,  1.0705557 ],
        [-0.16251644, -2.1981363 , -0.32122812,  3.5041664 , -0.564827  ],
        [-0.0923591 ,  0.32887274,  0.93221086, -1.7460452 , -0.12983538],
        [-0.80078787,  2.3014646 , -0.43455797, -2.7002316 , -0.25145605],
        [ 0.35079497, -1.425764  ,  0.14472388, -0.25879452, -0.4707508 ],
        [-0.34118485, -0.7758662 , -1.1711121 ,  0.8148826 ,  0.19521472],
        [-0.19109583, -0.44091418,  0.16165051,  0.44412947,  1.0168439 ]],
       dtype=float32),
 array([ 0.9159022, -2.3754754, -1.6497273, -1.6801256,  1.0722076],
       dtype=float32),
 array([[-0.30650178, -0.82059425, -0.8845178 ,  0.6198854 ,  0.01670245,
          0.91658044, -0.583803  ,  0.61051273,  1.0977648 , -0.2929295 ],
        [-0.11417517,  0.7498607 ,  0.80962944, -0.62643665,  0.0893286 ,
         -0.58019584,  0.7380129 , -0.62639546, -0.59378356, -0.14802209],
        [ 0.44325912,  0.91800797,  0.5974282 , -0.03354135, 

In [32]:
model_autompg.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_autompg_rmsprop_4th.h5')

In [33]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, adam2, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_13 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_20 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_14 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 119us/step - loss: 0.0064 - val_loss: 0.0039
Epoch 70/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0062 - val_loss: 0.0037
Epoch 71/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0063 - val_loss: 0.0035
Epoch 72/1000
143/143 [==============================] - 0s 125us/step - loss: 0.0062 - val_loss: 0.0036
Epoch 73/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0060 - val_loss: 0.0041
Epoch 74/1000
143/143 [==============================] - 0s 120us/step - loss: 0.0061 - val_loss: 0.0043
Epoch 75/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0062 - val_loss: 0.0041
Epoch 76/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0061 - val_loss: 0.0038
Epoch 77/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0060 - val_loss: 0.0041
Epoch 78/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0037
Epoch 147/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0048 - val_loss: 0.0034
Epoch 148/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0050 - val_loss: 0.0057
Epoch 149/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0036
Epoch 150/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0051 - val_loss: 0.0054
Epoch 151/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0051 - val_loss: 0.0045
Epoch 152/1000
143/143 [==============================] - 0s 121us/step - loss: 0.0052 - val_loss: 0.0061
Epoch 153/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0044 - val_loss: 0.0042
Epoch 154/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0058 - val_loss: 0.0049
Epoch 155/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0038 - val_loss: 0.0029
Epoch 224/1000
143/143 [==============================] - 0s 126us/step - loss: 0.0038 - val_loss: 0.0028
Epoch 225/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0038 - val_loss: 0.0038
Epoch 226/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0039 - val_loss: 0.0032
Epoch 227/1000
143/143 [==============================] - 0s 106us/step - loss: 0.0038 - val_loss: 0.0040
Epoch 228/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0043 - val_loss: 0.0044
Epoch 229/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0041 - val_loss: 0.0031
Epoch 230/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0038 - val_loss: 0.0041
Epoch 231/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0037 - val_loss: 0.0029
Epoch 232/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 112us/step - loss: 0.0038 - val_loss: 0.0067
Epoch 301/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0058 - val_loss: 0.0068
Epoch 302/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0062 - val_loss: 0.0035
Epoch 303/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0043 - val_loss: 0.0059
Epoch 304/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0053 - val_loss: 0.0031
Epoch 305/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0039 - val_loss: 0.0032
Epoch 306/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0035 - val_loss: 0.0032
Epoch 307/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0036 - val_loss: 0.0026
Epoch 308/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0034 - val_loss: 0.0029
Epoch 309/1000
143/143 [==============================] - 0s 

...

143/143 [==============================] - 0s 105us/step - loss: 0.0017 - val_loss: 0.0048
Epoch 763/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0022 - val_loss: 0.0032
Epoch 764/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0018 - val_loss: 0.0031
Epoch 765/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0019 - val_loss: 0.0031
Epoch 766/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0021 - val_loss: 0.0046
Epoch 767/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0019 - val_loss: 0.0033
Epoch 768/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0017 - val_loss: 0.0032
Epoch 769/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0033
Epoch 770/1000
143/143 [==============================] - 0s 113us/step - loss: 0.0024 - val_loss: 0.0030
Epoch 771/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 116us/step - loss: 0.0015 - val_loss: 0.0028
Epoch 840/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0016 - val_loss: 0.0028
Epoch 841/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0017 - val_loss: 0.0028
Epoch 842/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0019 - val_loss: 0.0030
Epoch 843/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0016 - val_loss: 0.0035
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0031
Epoch 845/1000
143/143 [==============================] - 0s 114us/step - loss: 0.0015 - val_loss: 0.0030
Epoch 846/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0014 - val_loss: 0.0027
Epoch 847/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0028
Epoch 848/1000
143/143 [==============================] - 0s 

143/143 [==============================] - 0s 119us/step - loss: 0.0016 - val_loss: 0.0040
Epoch 917/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0014 - val_loss: 0.0037
Epoch 918/1000
143/143 [==============================] - 0s 109us/step - loss: 0.0016 - val_loss: 0.0029
Epoch 919/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0015 - val_loss: 0.0028
Epoch 920/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0013 - val_loss: 0.0031
Epoch 921/1000
143/143 [==============================] - 0s 118us/step - loss: 0.0013 - val_loss: 0.0036
Epoch 922/1000
143/143 [==============================] - 0s 119us/step - loss: 0.0014 - val_loss: 0.0030
Epoch 923/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0013 - val_loss: 0.0027
Epoch 924/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0014 - val_loss: 0.0034
Epoch 925/1000
143/143 [==============================] - 0s 1

143/143 [==============================] - 0s 105us/step - loss: 0.0012 - val_loss: 0.0036
Epoch 994/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0011 - val_loss: 0.0032
Epoch 995/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0012 - val_loss: 0.0041
Epoch 996/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0062
Epoch 997/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0021 - val_loss: 0.0040
Epoch 998/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0015 - val_loss: 0.0032
Epoch 999/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0013 - val_loss: 0.0032
Epoch 1000/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0015 - val_loss: 0.0035
0.006766484584659338


[array([[ 1.09280849e+00, -4.69725206e-02,  9.37820449e-02,
          2.57516086e-01, -6.49350047e-01],
        [ 2.28303820e-01, -3.09773743e-01, -8.27465832e-01,
         -2.30400875e-01,  8.95458087e-02],
        [-4.38138664e-01, -3.19319099e-01,  2.32274100e-01,
         -6.41566992e-01, -3.93269286e-02],
        [-3.20079923e-01,  3.61049771e-01,  9.77661967e-01,
         -6.62049294e-01,  1.07967332e-01],
        [-7.69383609e-02, -3.24509703e-02,  6.49848878e-02,
          3.32513750e-02, -7.16576055e-02],
        [ 3.68470073e-01, -6.69867575e-01, -8.99207473e-01,
          3.47276688e-01, -4.22219038e-01],
        [-8.05617452e-01, -3.33464295e-01,  1.42124921e-01,
          1.10584922e-01, -3.86808425e-01],
        [ 3.47541794e-02,  1.45341799e-01,  2.58539826e-01,
          2.16535255e-01, -1.41095340e-01],
        [-4.11290348e-01, -8.21884274e-01,  1.32257676e+00,
          5.43036163e-01, -8.87208998e-01],
        [ 1.43000782e+00, -1.12514324e-01, -3.90260994e-01,
    

In [34]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_adam_4th.h5')

In [35]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, sgd, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_15 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_23 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_16 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 105us/step - loss: 0.0906 - val_loss: 0.0867
Epoch 70/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0905 - val_loss: 0.0867
Epoch 71/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0904 - val_loss: 0.0866
Epoch 72/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0904 - val_loss: 0.0866
Epoch 73/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0903 - val_loss: 0.0865
Epoch 74/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0902 - val_loss: 0.0865
Epoch 75/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0901 - val_loss: 0.0864
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0900 - val_loss: 0.0864
Epoch 77/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0899 - val_loss: 0.0863
Epoch 78/1000
143/143 [==============================] - 0s 97us/step - lo

143/143 [==============================] - 0s 112us/step - loss: 0.0846 - val_loss: 0.0833
Epoch 148/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0846 - val_loss: 0.0832
Epoch 149/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0845 - val_loss: 0.0832
Epoch 150/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0844 - val_loss: 0.0832
Epoch 151/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0844 - val_loss: 0.0831
Epoch 152/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0843 - val_loss: 0.0831
Epoch 153/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0842 - val_loss: 0.0831
Epoch 154/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0841 - val_loss: 0.0830
Epoch 155/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0841 - val_loss: 0.0830
Epoch 156/1000
143/143 [==============================] - 0s 84us/ste

Epoch 225/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0799 - val_loss: 0.0807
Epoch 226/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0798 - val_loss: 0.0806
Epoch 227/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0797 - val_loss: 0.0806
Epoch 228/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0797 - val_loss: 0.0806
Epoch 229/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0796 - val_loss: 0.0806
Epoch 230/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0796 - val_loss: 0.0805
Epoch 231/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0795 - val_loss: 0.0805
Epoch 232/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0795 - val_loss: 0.0805
Epoch 233/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0794 - val_loss: 0.0804
Epoch 234/1000
143/143 [==============================]

Epoch 303/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0760 - val_loss: 0.0786
Epoch 304/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0759 - val_loss: 0.0786
Epoch 305/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0759 - val_loss: 0.0786
Epoch 306/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0759 - val_loss: 0.0786
Epoch 307/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0758 - val_loss: 0.0785
Epoch 308/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0758 - val_loss: 0.0785
Epoch 309/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0757 - val_loss: 0.0785
Epoch 310/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0757 - val_loss: 0.0785
Epoch 311/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0756 - val_loss: 0.0785
Epoch 312/1000
143/143 [==============================]

...

Epoch 692/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0647 - val_loss: 0.0731
Epoch 693/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0647 - val_loss: 0.0731
Epoch 694/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0646 - val_loss: 0.0731
Epoch 695/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0646 - val_loss: 0.0731
Epoch 696/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0646 - val_loss: 0.0731
Epoch 697/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0646 - val_loss: 0.0731
Epoch 698/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0646 - val_loss: 0.0730
Epoch 699/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0645 - val_loss: 0.0730
Epoch 700/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0645 - val_loss: 0.0730
Epoch 701/1000
143/143 [==============================]

143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0725
Epoch 770/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0724
Epoch 771/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0724
Epoch 772/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0724
Epoch 773/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0633 - val_loss: 0.0724
Epoch 774/1000
143/143 [==============================] - 0s 77us/step - loss: 0.0633 - val_loss: 0.0724
Epoch 775/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0632 - val_loss: 0.0724
Epoch 776/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0632 - val_loss: 0.0724
Epoch 777/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0632 - val_loss: 0.0724
Epoch 778/1000
143/143 [==============================] - 0s 105us/ste

143/143 [==============================] - 0s 91us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 848/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 849/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 850/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 851/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 852/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 853/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0621 - val_loss: 0.0718
Epoch 854/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0620 - val_loss: 0.0718
Epoch 855/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0620 - val_loss: 0.0718
Epoch 856/1000
143/143 [==============================] - 0s 91us/step

143/143 [==============================] - 0s 91us/step - loss: 0.0611 - val_loss: 0.0713
Epoch 926/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0611 - val_loss: 0.0713
Epoch 927/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0611 - val_loss: 0.0713
Epoch 928/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0611 - val_loss: 0.0713
Epoch 929/1000
143/143 [==============================] - 0s 84us/step - loss: 0.0610 - val_loss: 0.0713
Epoch 930/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0610 - val_loss: 0.0713
Epoch 931/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0610 - val_loss: 0.0712
Epoch 932/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0610 - val_loss: 0.0712
Epoch 933/1000
143/143 [==============================] - 0s 89us/step - loss: 0.0610 - val_loss: 0.0712
Epoch 934/1000
143/143 [==============================] - 0s 91us/step

[array([[-0.45197   , -0.18194647,  0.19040312,  0.13549015, -0.13787475],
        [-0.35525405, -0.08197729,  0.26338264, -0.04371401,  0.24996547],
        [ 0.3542493 , -0.4040447 , -0.05913693,  0.05483595,  0.15408912],
        [ 0.00760729, -0.1966862 , -0.2202771 , -0.40061685, -0.2770086 ],
        [ 0.4583157 ,  0.4571072 , -0.14390117,  0.10478602,  0.04813117],
        [-0.42968318, -0.43324718,  0.29981327,  0.22062139,  0.25859982],
        [ 0.43665075,  0.02570491, -0.00632085,  0.03287093, -0.24762739],
        [ 0.437256  , -0.04783225,  0.22768451, -0.41456124, -0.43501008],
        [ 0.27832022,  0.23796462,  0.05645671, -0.0660567 , -0.19204684],
        [-0.36083192, -0.09220301,  0.24773034,  0.06004743,  0.26164725],
        [-0.34205052,  0.17353325, -0.13583308,  0.3771883 , -0.23250557],
        [-0.26113462,  0.15293707, -0.28178668,  0.20950551,  0.32894343],
        [ 0.06508092,  0.14980057, -0.23753937, -0.12493879, -0.09655069],
        [-0.12638728,  0.

In [36]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_sgd_4th.h5')

In [37]:
model_automobile, history_automobile, evaluate_automobile=NN_model_structure_regression(X_train_automobile, X_val_automobile, Y_train_automobile, Y_val_automobile, RMSprop, 32, 1000, x_test_automobile, y_test_automobile)
model_automobile.get_weights()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_25 (Dense)             (None, 5)                 115       
_________________________________________________________________
activation_17 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_26 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_18 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 1)                 11        
Total params: 186
Trainable params: 186
Non-trainable params: 0
_________________________________________________________________
Train on 143 samples, validate on 31 samples
Epoch 1/1000
143/143 [==============================] - 0s 1ms/step - loss: 0

143/143 [==============================] - 0s 111us/step - loss: 0.0138 - val_loss: 0.0134
Epoch 70/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0101 - val_loss: 0.0050
Epoch 71/1000
143/143 [==============================] - 0s 111us/step - loss: 0.0105 - val_loss: 0.0085
Epoch 72/1000
143/143 [==============================] - 0s 107us/step - loss: 0.0093 - val_loss: 0.0050
Epoch 73/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0104 - val_loss: 0.0084
Epoch 74/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0106 - val_loss: 0.0091
Epoch 75/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0094 - val_loss: 0.0110
Epoch 76/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0099 - val_loss: 0.0070
Epoch 77/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0148 - val_loss: 0.0079
Epoch 78/1000
143/143 [==============================] - 0s 112us/step 

Epoch 147/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0080 - val_loss: 0.0068
Epoch 148/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0084 - val_loss: 0.0046
Epoch 149/1000
143/143 [==============================] - 0s 96us/step - loss: 0.0086 - val_loss: 0.0050
Epoch 150/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0079 - val_loss: 0.0051
Epoch 151/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0080 - val_loss: 0.0049
Epoch 152/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0069 - val_loss: 0.0057
Epoch 153/1000
143/143 [==============================] - 0s 108us/step - loss: 0.0073 - val_loss: 0.0056
Epoch 154/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0071 - val_loss: 0.0064
Epoch 155/1000
143/143 [==============================] - 0s 112us/step - loss: 0.0100 - val_loss: 0.0106
Epoch 156/1000
143/143 [========================

143/143 [==============================] - 0s 105us/step - loss: 0.0068 - val_loss: 0.0055
Epoch 225/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0063 - val_loss: 0.0081
Epoch 226/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0076 - val_loss: 0.0101
Epoch 227/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0075 - val_loss: 0.0068
Epoch 228/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0066 - val_loss: 0.0091
Epoch 229/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0066 - val_loss: 0.0046
Epoch 230/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0067 - val_loss: 0.0071
Epoch 231/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0069 - val_loss: 0.0040
Epoch 232/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0063 - val_loss: 0.0055
Epoch 233/1000
143/143 [==============================] - 0s 9

143/143 [==============================] - 0s 104us/step - loss: 0.0054 - val_loss: 0.0038
Epoch 302/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0059 - val_loss: 0.0052
Epoch 303/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0064 - val_loss: 0.0087
Epoch 304/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0063 - val_loss: 0.0047
Epoch 305/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0064 - val_loss: 0.0048
Epoch 306/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0061 - val_loss: 0.0062
Epoch 307/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0064 - val_loss: 0.0072
Epoch 308/1000
143/143 [==============================] - 0s 97us/step - loss: 0.0056 - val_loss: 0.0045
Epoch 309/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0062 - val_loss: 0.0048
Epoch 310/1000
143/143 [==============================] - 0s 105u

...

143/143 [==============================] - 0s 98us/step - loss: 0.0054 - val_loss: 0.0028
Epoch 765/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0023 - val_loss: 0.0037
Epoch 766/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0031 - val_loss: 0.0035
Epoch 767/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0027 - val_loss: 0.0048
Epoch 768/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0046 - val_loss: 0.0032
Epoch 769/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0025 - val_loss: 0.0030
Epoch 770/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0029 - val_loss: 0.0039
Epoch 771/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0025 - val_loss: 0.0036
Epoch 772/1000
143/143 [==============================] - 0s 104us/step - loss: 0.0033 - val_loss: 0.0041
Epoch 773/1000
143/143 [==============================] - 0s 98us

Epoch 842/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 843/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0023 - val_loss: 0.0035
Epoch 844/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0021 - val_loss: 0.0030
Epoch 845/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0034 - val_loss: 0.0043
Epoch 846/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0029 - val_loss: 0.0034
Epoch 847/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0020 - val_loss: 0.0031
Epoch 848/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0029 - val_loss: 0.0053
Epoch 849/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0031 - val_loss: 0.0039
Epoch 850/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0030 - val_loss: 0.0031
Epoch 851/1000
143/143 [==============================

143/143 [==============================] - 0s 98us/step - loss: 0.0019 - val_loss: 0.0032
Epoch 920/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0026 - val_loss: 0.0035
Epoch 921/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0029 - val_loss: 0.0032
Epoch 922/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0028 - val_loss: 0.0032
Epoch 923/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0023 - val_loss: 0.0039
Epoch 924/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0019 - val_loss: 0.0031
Epoch 925/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0026 - val_loss: 0.0034
Epoch 926/1000
143/143 [==============================] - 0s 91us/step - loss: 0.0021 - val_loss: 0.0037
Epoch 927/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0021 - val_loss: 0.0037
Epoch 928/1000
143/143 [==============================] - 0s 105us/s

Epoch 997/1000
143/143 [==============================] - 0s 105us/step - loss: 0.0020 - val_loss: 0.0044
Epoch 998/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0024 - val_loss: 0.0036
Epoch 999/1000
143/143 [==============================] - ETA: 0s - loss: 0.003 - 0s 98us/step - loss: 0.0035 - val_loss: 0.0037
Epoch 1000/1000
143/143 [==============================] - 0s 98us/step - loss: 0.0021 - val_loss: 0.0049
0.010828805156052113


[array([[ 9.78599906e-01, -7.54701614e-01, -1.16646096e-01,
          8.80859137e-01, -3.23170692e-01],
        [ 1.94833159e+00, -7.75677741e-01, -1.28074750e-01,
         -5.75051606e-01, -1.37827128e-01],
        [-1.29131877e+00,  3.37375343e-01,  7.63800740e-02,
         -1.50479507e+00,  2.95576423e-01],
        [ 4.29194957e-01, -1.86303467e-03,  2.30076775e-01,
         -8.18101704e-01, -3.82067412e-02],
        [-2.01021686e-01,  2.41901651e-02,  1.15593076e-01,
         -4.86807317e-01,  1.03044331e-01],
        [ 6.85561538e-01, -6.68647289e-01,  4.88949344e-02,
          1.62072980e+00, -3.93864453e-01],
        [ 9.28017557e-01, -8.50471199e-01,  2.18809485e-01,
         -7.80623853e-01, -3.97662371e-02],
        [ 8.56747985e-01, -1.38361007e-02, -1.24440596e-01,
         -2.48450376e-02, -1.62792876e-01],
        [-1.96086913e-01,  2.32188523e-01, -4.16543558e-02,
         -3.27066451e-01, -3.00939977e-01],
        [ 1.69159055e+00, -4.62307036e-01, -2.18834877e-02,
    

In [38]:
model_automobile.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_automobile_rmsprop_4th.h5')

In [39]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, adam2, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_28 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_19 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_29 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_20 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 35.505

Epoch 71/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0136 - val_loss: 0.0077
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0138 - val_loss: 0.0067
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0134 - val_loss: 0.0054
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0122 - val_loss: 0.0043
Epoch 75/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0107 - val_loss: 0.0036
Epoch 76/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0092 - val_loss: 0.0034
Epoch 77/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0079 - val_loss: 0.0037
Epoch 78/1000
32/32 [==============================] - 0s 126us/step - loss: 0.0071 - val_loss: 0.0045
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0069 - val_loss: 0.0056
Epoch 80/1000
32/32 [==============================] - 0s 187us/step - lo

Epoch 151/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0018
Epoch 152/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0047 - val_loss: 0.0018
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0047 - val_loss: 0.0018
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0047 - val_loss: 0.0018
Epoch 155/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0017
Epoch 156/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0017
Epoch 157/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0017
Epoch 158/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0046 - val_loss: 0.0017
Epoch 159/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0046 - val_loss: 0.0017
Epoch 160/1000
32/32 [==============================] - 0s 156us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 231/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 232/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 233/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 234/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 235/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 236/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 237/1000
32/32 [==============================] - 0s 130us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 238/1000
32/32 [==============================] - 0s 123us/step - loss: 0.0035 - val_loss: 0.0012
Epoch 239/1000
32/32 [==============================] - 0s 156us

Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0028 - val_loss: 8.8520e-04
Epoch 309/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0028 - val_loss: 8.8185e-04
Epoch 310/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 8.7852e-04
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0028 - val_loss: 8.7520e-04
Epoch 312/1000
32/32 [==============================] - 0s 186us/step - loss: 0.0028 - val_loss: 8.7191e-04
Epoch 313/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 8.6863e-04
Epoch 314/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0028 - val_loss: 8.6537e-04
Epoch 315/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0028 - val_loss: 8.6213e-04
Epoch 316/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0028 - val_loss: 8.5890e-04
Epoch 317/1000
32/32 [======

...

Epoch 764/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0011 - val_loss: 2.5210e-04
Epoch 765/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 2.5156e-04
Epoch 766/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0011 - val_loss: 2.5102e-04
Epoch 767/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0011 - val_loss: 2.5049e-04
Epoch 768/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0010 - val_loss: 2.4995e-04
Epoch 769/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 2.4943e-04
Epoch 770/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 2.4889e-04
Epoch 771/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0010 - val_loss: 2.4836e-04
Epoch 772/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0010 - val_loss: 2.4784e-04
Epoch 773/1000
32/32 [======

Epoch 839/1000
32/32 [==============================] - 0s 125us/step - loss: 8.9621e-04 - val_loss: 2.1527e-04
Epoch 840/1000
32/32 [==============================] - 0s 125us/step - loss: 8.9416e-04 - val_loss: 2.1482e-04
Epoch 841/1000
32/32 [==============================] - 0s 125us/step - loss: 8.9211e-04 - val_loss: 2.1437e-04
Epoch 842/1000
32/32 [==============================] - 0s 125us/step - loss: 8.9008e-04 - val_loss: 2.1394e-04
Epoch 843/1000
32/32 [==============================] - 0s 125us/step - loss: 8.8803e-04 - val_loss: 2.1349e-04
Epoch 844/1000
32/32 [==============================] - 0s 155us/step - loss: 8.8601e-04 - val_loss: 2.1304e-04
Epoch 845/1000
32/32 [==============================] - 0s 156us/step - loss: 8.8398e-04 - val_loss: 2.1259e-04
Epoch 846/1000
32/32 [==============================] - 0s 94us/step - loss: 8.8195e-04 - val_loss: 2.1215e-04
Epoch 847/1000
32/32 [==============================] - 0s 125us/step - loss: 8.7992e-04 - val_loss: 2.11

Epoch 913/1000
32/32 [==============================] - 0s 125us/step - loss: 7.5155e-04 - val_loss: 1.8489e-04
Epoch 914/1000
32/32 [==============================] - 0s 125us/step - loss: 7.4969e-04 - val_loss: 1.8450e-04
Epoch 915/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4783e-04 - val_loss: 1.8414e-04
Epoch 916/1000
32/32 [==============================] - 0s 133us/step - loss: 7.4597e-04 - val_loss: 1.8377e-04
Epoch 917/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4412e-04 - val_loss: 1.8339e-04
Epoch 918/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4226e-04 - val_loss: 1.8302e-04
Epoch 919/1000
32/32 [==============================] - 0s 156us/step - loss: 7.4041e-04 - val_loss: 1.8264e-04
Epoch 920/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3856e-04 - val_loss: 1.8228e-04
Epoch 921/1000
32/32 [==============================] - 0s 156us/step - loss: 7.3671e-04 - val_loss: 1.8

Epoch 987/1000
32/32 [==============================] - 0s 125us/step - loss: 6.2002e-04 - val_loss: 1.5938e-04
Epoch 988/1000
32/32 [==============================] - 0s 125us/step - loss: 6.1833e-04 - val_loss: 1.5906e-04
Epoch 989/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1665e-04 - val_loss: 1.5875e-04
Epoch 990/1000
32/32 [==============================] - 0s 156us/step - loss: 6.1496e-04 - val_loss: 1.5844e-04
Epoch 991/1000
32/32 [==============================] - 0s 155us/step - loss: 6.1329e-04 - val_loss: 1.5813e-04
Epoch 992/1000
32/32 [==============================] - 0s 125us/step - loss: 6.1161e-04 - val_loss: 1.5782e-04
Epoch 993/1000
32/32 [==============================] - 0s 136us/step - loss: 6.0993e-04 - val_loss: 1.5751e-04
Epoch 994/1000
32/32 [==============================] - 0s 125us/step - loss: 6.0826e-04 - val_loss: 1.5720e-04
Epoch 995/1000
32/32 [==============================] - 0s 156us/step - loss: 6.0659e-04 - val_loss: 1.5

[array([[ 1.1976997 ,  1.5927104 , -1.1752906 , -1.2031064 , -1.900142  ],
        [-0.34487632, -1.3794736 , -0.08579582,  0.4584256 ,  1.3568879 ],
        [-0.27764204, -0.22252198, -0.5472197 ,  0.77625304, -0.1435791 ]],
       dtype=float32),
 array([ 0.19651635, -0.64234316, -0.5309899 , -0.8096226 ,  0.5336856 ],
       dtype=float32),
 array([[-0.2966159 ,  0.1727865 ,  0.43303227,  0.21931909, -0.32757246,
         -0.12056155,  0.4277477 ,  0.6415886 ,  0.19594418,  0.32523718],
        [-0.3642033 ,  0.05506884,  0.5542948 , -0.20617746, -0.16246602,
          0.63586855,  0.14169088,  0.35682434,  0.26779062,  0.3255594 ],
        [ 0.29197773,  0.5702987 , -0.21816432,  0.50399613,  0.3267495 ,
         -0.04499248, -0.1804116 ,  0.7610589 , -0.17289652, -0.33659947],
        [ 0.7045552 ,  0.5131875 , -0.7573231 ,  0.6203033 , -0.24380857,
          0.42935875, -0.00260475,  0.30407017,  0.00347694,  0.22323215],
        [ 0.05688379,  0.22905153,  0.16217014,  0.2994501

In [40]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_adam_4th.h5')

In [41]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, sgd, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_21 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_32 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_22 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 6ms/step - loss: 35.438

Epoch 70/1000
32/32 [==============================] - 0s 156us/step - loss: 34.9004 - val_loss: 30.9967
Epoch 71/1000
32/32 [==============================] - 0s 156us/step - loss: 34.8916 - val_loss: 30.9887
Epoch 72/1000
32/32 [==============================] - 0s 156us/step - loss: 34.8829 - val_loss: 30.9808
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8741 - val_loss: 30.9728
Epoch 74/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8653 - val_loss: 30.9649
Epoch 75/1000
32/32 [==============================] - 0s 93us/step - loss: 34.8566 - val_loss: 30.9569
Epoch 76/1000
32/32 [==============================] - 0s 93us/step - loss: 34.8478 - val_loss: 30.9490
Epoch 77/1000
32/32 [==============================] - 0s 124us/step - loss: 34.8390 - val_loss: 30.9410
Epoch 78/1000
32/32 [==============================] - 0s 125us/step - loss: 34.8303 - val_loss: 30.9331
Epoch 79/1000
32/32 [==============================] - 0s

Epoch 148/1000
32/32 [==============================] - 0s 94us/step - loss: 34.2221 - val_loss: 30.3804
Epoch 149/1000
32/32 [==============================] - 0s 125us/step - loss: 34.2135 - val_loss: 30.3726
Epoch 150/1000
32/32 [==============================] - 0s 125us/step - loss: 34.2049 - val_loss: 30.3647
Epoch 151/1000
32/32 [==============================] - 0s 125us/step - loss: 34.1963 - val_loss: 30.3569
Epoch 152/1000
32/32 [==============================] - 0s 93us/step - loss: 34.1877 - val_loss: 30.3491
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 34.1791 - val_loss: 30.3412
Epoch 154/1000
32/32 [==============================] - 0s 125us/step - loss: 34.1705 - val_loss: 30.3334
Epoch 155/1000
32/32 [==============================] - 0s 127us/step - loss: 34.1619 - val_loss: 30.3255
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 34.1533 - val_loss: 30.3177
Epoch 157/1000
32/32 [==========================

Epoch 226/1000
32/32 [==============================] - 0s 125us/step - loss: 33.5557 - val_loss: 29.7728
Epoch 227/1000
32/32 [==============================] - 0s 125us/step - loss: 33.5472 - val_loss: 29.7651
Epoch 228/1000
32/32 [==============================] - 0s 94us/step - loss: 33.5388 - val_loss: 29.7573
Epoch 229/1000
32/32 [==============================] - 0s 125us/step - loss: 33.5303 - val_loss: 29.7496
Epoch 230/1000
32/32 [==============================] - 0s 125us/step - loss: 33.5218 - val_loss: 29.7419
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 33.5134 - val_loss: 29.7341
Epoch 232/1000
32/32 [==============================] - 0s 124us/step - loss: 33.5049 - val_loss: 29.7264
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 33.4964 - val_loss: 29.7187
Epoch 234/1000
32/32 [==============================] - 0s 125us/step - loss: 33.4880 - val_loss: 29.7109
Epoch 235/1000
32/32 [=========================

Epoch 304/1000
32/32 [==============================] - 0s 125us/step - loss: 32.9000 - val_loss: 29.1731
Epoch 305/1000
32/32 [==============================] - 0s 93us/step - loss: 32.8917 - val_loss: 29.1654
Epoch 306/1000
32/32 [==============================] - 0s 125us/step - loss: 32.8834 - val_loss: 29.1578
Epoch 307/1000
32/32 [==============================] - 0s 124us/step - loss: 32.8750 - val_loss: 29.1502
Epoch 308/1000
32/32 [==============================] - 0s 125us/step - loss: 32.8667 - val_loss: 29.1425
Epoch 309/1000
32/32 [==============================] - 0s 124us/step - loss: 32.8584 - val_loss: 29.1349
Epoch 310/1000
32/32 [==============================] - 0s 94us/step - loss: 32.8500 - val_loss: 29.1272
Epoch 311/1000
32/32 [==============================] - 0s 125us/step - loss: 32.8417 - val_loss: 29.1196
Epoch 312/1000
32/32 [==============================] - 0s 155us/step - loss: 32.8334 - val_loss: 29.1120
Epoch 313/1000
32/32 [==========================

...

Epoch 694/1000
32/32 [==============================] - 0s 94us/step - loss: 29.7518 - val_loss: 26.2714
Epoch 695/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7439 - val_loss: 26.2641
Epoch 696/1000
32/32 [==============================] - 0s 93us/step - loss: 29.7361 - val_loss: 26.2568
Epoch 697/1000
32/32 [==============================] - 0s 94us/step - loss: 29.7282 - val_loss: 26.2496
Epoch 698/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7204 - val_loss: 26.2423
Epoch 699/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7126 - val_loss: 26.2351
Epoch 700/1000
32/32 [==============================] - 0s 125us/step - loss: 29.7047 - val_loss: 26.2278
Epoch 701/1000
32/32 [==============================] - 0s 125us/step - loss: 29.6969 - val_loss: 26.2205
Epoch 702/1000
32/32 [==============================] - 0s 125us/step - loss: 29.6891 - val_loss: 26.2133
Epoch 703/1000
32/32 [===========================

Epoch 772/1000
32/32 [==============================] - 0s 93us/step - loss: 29.1430 - val_loss: 25.7071
Epoch 773/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1352 - val_loss: 25.6999
Epoch 774/1000
32/32 [==============================] - 0s 93us/step - loss: 29.1275 - val_loss: 25.6927
Epoch 775/1000
32/32 [==============================] - 0s 93us/step - loss: 29.1197 - val_loss: 25.6855
Epoch 776/1000
32/32 [==============================] - 0s 94us/step - loss: 29.1120 - val_loss: 25.6783
Epoch 777/1000
32/32 [==============================] - 0s 125us/step - loss: 29.1042 - val_loss: 25.6711
Epoch 778/1000
32/32 [==============================] - 0s 125us/step - loss: 29.0964 - val_loss: 25.6639
Epoch 779/1000
32/32 [==============================] - 0s 156us/step - loss: 29.0887 - val_loss: 25.6567
Epoch 780/1000
32/32 [==============================] - 0s 93us/step - loss: 29.0809 - val_loss: 25.6495
Epoch 781/1000
32/32 [=============================

Epoch 850/1000
32/32 [==============================] - 0s 125us/step - loss: 28.5399 - val_loss: 25.1474
Epoch 851/1000
32/32 [==============================] - 0s 125us/step - loss: 28.5322 - val_loss: 25.1403
Epoch 852/1000
32/32 [==============================] - 0s 125us/step - loss: 28.5245 - val_loss: 25.1331
Epoch 853/1000
32/32 [==============================] - 0s 125us/step - loss: 28.5168 - val_loss: 25.1260
Epoch 854/1000
32/32 [==============================] - 0s 93us/step - loss: 28.5091 - val_loss: 25.1188
Epoch 855/1000
32/32 [==============================] - 0s 93us/step - loss: 28.5014 - val_loss: 25.1117
Epoch 856/1000
32/32 [==============================] - 0s 125us/step - loss: 28.4937 - val_loss: 25.1046
Epoch 857/1000
32/32 [==============================] - 0s 125us/step - loss: 28.4860 - val_loss: 25.0974
Epoch 858/1000
32/32 [==============================] - 0s 125us/step - loss: 28.4783 - val_loss: 25.0903
Epoch 859/1000
32/32 [==========================

Epoch 928/1000
32/32 [==============================] - 0s 94us/step - loss: 27.9419 - val_loss: 24.5921
Epoch 929/1000
32/32 [==============================] - 0s 125us/step - loss: 27.9343 - val_loss: 24.5850
Epoch 930/1000
32/32 [==============================] - 0s 125us/step - loss: 27.9266 - val_loss: 24.5779
Epoch 931/1000
32/32 [==============================] - 0s 125us/step - loss: 27.9190 - val_loss: 24.5708
Epoch 932/1000
32/32 [==============================] - 0s 125us/step - loss: 27.9114 - val_loss: 24.5637
Epoch 933/1000
32/32 [==============================] - 0s 125us/step - loss: 27.9037 - val_loss: 24.5566
Epoch 934/1000
32/32 [==============================] - 0s 125us/step - loss: 27.8961 - val_loss: 24.5495
Epoch 935/1000
32/32 [==============================] - 0s 156us/step - loss: 27.8885 - val_loss: 24.5424
Epoch 936/1000
32/32 [==============================] - 0s 125us/step - loss: 27.8808 - val_loss: 24.5353
Epoch 937/1000
32/32 [=========================

[array([[-0.8305471 ,  0.7705578 ,  0.27940595,  0.7556379 ,  0.7294787 ],
        [-0.03815834,  0.532696  ,  0.5562497 , -0.7589856 ,  0.22267678],
        [-0.28797114,  0.74631906,  0.44201115,  0.10418889, -0.5666954 ]],
       dtype=float32),
 array([ 0.05293802,  0.0342578 , -0.0532173 , -0.00606847, -0.15667406],
       dtype=float32),
 array([[-0.1916619 , -0.28779572, -0.00834114,  0.4254611 , -0.50992805,
         -0.1459388 , -0.24729462, -0.54075277, -0.4517089 ,  0.5488235 ],
        [ 0.5043219 , -0.53915775,  0.13444424, -0.5689076 , -0.00989301,
          0.30515406,  0.37299782,  0.04431949,  0.569056  ,  0.43702424],
        [ 0.04747498, -0.42674494, -0.20302562, -0.51453274,  0.31469244,
         -0.26319045, -0.46886098,  0.30837822, -0.19095482, -0.37331492],
        [-0.1821902 ,  0.13624719, -0.5951666 , -0.20916334, -0.17353888,
         -0.28358448,  0.4401992 ,  0.26028746,  0.10183384,  0.11201303],
        [-0.3423488 ,  0.588493  , -0.61355066,  0.5780125

In [42]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_sgd_4th.h5')

In [43]:
model_challenger, history_challenger, evaluate_challenger=NN_model_structure_regression(X_train_challenger, X_val_challenger, Y_train_challenger, Y_val_challenger, RMSprop, 32, 1000, x_test_challenger, y_test_challenger)
model_challenger.get_weights()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 5)                 20        
_________________________________________________________________
activation_23 (Activation)   (None, 5)                 0         
_________________________________________________________________
dense_35 (Dense)             (None, 10)                60        
_________________________________________________________________
activation_24 (Activation)   (None, 10)                0         
_________________________________________________________________
dense_36 (Dense)             (None, 1)                 11        
Total params: 91
Trainable params: 91
Non-trainable params: 0
_________________________________________________________________
Train on 32 samples, validate on 7 samples
Epoch 1/1000
32/32 [==============================] - 0s 7ms/step - loss: 36.445

Epoch 71/1000
32/32 [==============================] - 0s 150us/step - loss: 0.1283 - val_loss: 0.1188
Epoch 72/1000
32/32 [==============================] - 0s 125us/step - loss: 0.1304 - val_loss: 0.2411
Epoch 73/1000
32/32 [==============================] - 0s 125us/step - loss: 0.1343 - val_loss: 0.1118
Epoch 74/1000
32/32 [==============================] - 0s 156us/step - loss: 0.1245 - val_loss: 0.2242
Epoch 75/1000
32/32 [==============================] - 0s 125us/step - loss: 0.1201 - val_loss: 0.1040
Epoch 76/1000
32/32 [==============================] - 0s 94us/step - loss: 0.1085 - val_loss: 0.2015
Epoch 77/1000
32/32 [==============================] - 0s 155us/step - loss: 0.1043 - val_loss: 0.0989
Epoch 78/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0974 - val_loss: 0.1887
Epoch 79/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0963 - val_loss: 0.0959
Epoch 80/1000
32/32 [==============================] - 0s 125us/step - loss

Epoch 151/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0690 - val_loss: 0.0361
Epoch 152/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0669 - val_loss: 0.1193
Epoch 153/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0693 - val_loss: 0.0331
Epoch 154/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0616 - val_loss: 0.1028
Epoch 155/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0611 - val_loss: 0.0325
Epoch 156/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0552 - val_loss: 0.0932
Epoch 157/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0567 - val_loss: 0.0374
Epoch 158/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0558 - val_loss: 0.0961
Epoch 159/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0624 - val_loss: 0.0530
Epoch 160/1000
32/32 [==============================] - 0s 125us

Epoch 230/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0435 - val_loss: 0.0626
Epoch 231/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0506 - val_loss: 0.0367
Epoch 232/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0562 - val_loss: 0.0752
Epoch 233/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0641 - val_loss: 0.0400
Epoch 234/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0591 - val_loss: 0.0619
Epoch 235/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0526 - val_loss: 0.0257
Epoch 236/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0437 - val_loss: 0.0558
Epoch 237/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0413 - val_loss: 0.0209
Epoch 238/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0397 - val_loss: 0.0665
Epoch 239/1000
32/32 [==============================] - 0s 125us

Epoch 309/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0373 - val_loss: 0.0187
Epoch 310/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0345 - val_loss: 0.0503
Epoch 311/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0381 - val_loss: 0.0222
Epoch 312/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0406 - val_loss: 0.0642
Epoch 313/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0483 - val_loss: 0.0245
Epoch 314/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0475 - val_loss: 0.0616
Epoch 315/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0498 - val_loss: 0.0180
Epoch 316/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0423 - val_loss: 0.0469
Epoch 317/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0415 - val_loss: 0.0133
Epoch 318/1000
32/32 [==============================] - 0s 124us

...

Epoch 704/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0333 - val_loss: 0.0407
Epoch 705/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0389 - val_loss: 0.0351
Epoch 706/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0373 - val_loss: 0.0423
Epoch 707/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0362 - val_loss: 0.0286
Epoch 708/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0295 - val_loss: 0.0336
Epoch 709/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0277 - val_loss: 0.0234
Epoch 710/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0251 - val_loss: 0.0338
Epoch 711/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0279 - val_loss: 0.0254
Epoch 712/1000
32/32 [==============================] - 0s 187us/step - loss: 0.0289 - val_loss: 0.0382
Epoch 713/1000
32/32 [==============================] - 0s 156us

Epoch 783/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0261 - val_loss: 0.0263
Epoch 784/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0264 - val_loss: 0.0367
Epoch 785/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0308 - val_loss: 0.0265
Epoch 786/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0302 - val_loss: 0.0336
Epoch 787/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0328 - val_loss: 0.0203
Epoch 788/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0292 - val_loss: 0.0253
Epoch 789/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0300 - val_loss: 0.0147
Epoch 790/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0264 - val_loss: 0.0204
Epoch 791/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0276 - val_loss: 0.0140
Epoch 792/1000
32/32 [==============================] - 0s 125us

Epoch 862/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0252 - val_loss: 0.0190
Epoch 863/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0264 - val_loss: 0.0133
Epoch 864/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0245 - val_loss: 0.0194
Epoch 865/1000
32/32 [==============================] - 0s 124us/step - loss: 0.0273 - val_loss: 0.0179
Epoch 866/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0276 - val_loss: 0.0280
Epoch 867/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0325 - val_loss: 0.0279
Epoch 868/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0324 - val_loss: 0.0366
Epoch 869/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0339 - val_loss: 0.0303
Epoch 870/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0289 - val_loss: 0.0335
Epoch 871/1000
32/32 [==============================] - 0s 124us/

Epoch 941/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0348 - val_loss: 0.0283
Epoch 942/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0291 - val_loss: 0.0298
Epoch 943/1000
32/32 [==============================] - 0s 93us/step - loss: 0.0258 - val_loss: 0.0237
Epoch 944/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0209 - val_loss: 0.0278
Epoch 945/1000
32/32 [==============================] - 0s 156us/step - loss: 0.0211 - val_loss: 0.0264
Epoch 946/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0216 - val_loss: 0.0350
Epoch 947/1000
32/32 [==============================] - 0s 125us/step - loss: 0.0264 - val_loss: 0.0320
Epoch 948/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0279 - val_loss: 0.0366
Epoch 949/1000
32/32 [==============================] - 0s 94us/step - loss: 0.0312 - val_loss: 0.0260
Epoch 950/1000
32/32 [==============================] - 0s 93us/ste

[array([[-0.4121352 ,  0.9020908 ,  0.19666164,  0.1100288 ,  0.48310325],
        [-0.1291268 , -0.17443722, -0.6422074 ,  0.0727801 , -0.46893385],
        [-0.12209765,  0.18586026,  0.1493269 , -0.59487283, -0.4582646 ]],
       dtype=float32),
 array([-0.55472976, -0.44968447, -0.16833316, -0.2368574 ,  0.27591002],
       dtype=float32),
 array([[-7.90816769e-02, -3.41731519e-01,  2.18523636e-01,
         -2.42665172e-01, -5.45350969e-01, -1.30947500e-01,
          8.11292410e-01,  5.80937386e-01,  7.61799514e-01,
          3.39280039e-01],
        [-1.95925042e-01, -4.63499218e-01,  1.47924259e-01,
         -6.25007749e-01,  4.29646701e-01, -7.73511469e-01,
         -1.55248940e-01,  1.55250980e-02,  7.36702800e-01,
          1.32414132e-01],
        [ 6.99152410e-01,  7.89372772e-02,  5.51239610e-01,
         -2.50712276e-01,  2.95844018e-01, -2.67059356e-01,
         -4.67058122e-01,  2.76678443e-01, -4.49477017e-01,
          4.95610476e-01],
        [ 4.43712920e-01,  4.2517

In [44]:
model_challenger.save('C:\\Users\\user\\Desktop\\Thesis\\model_weights\\MLP_challenger_rmsprop_4th.h5')